In [1]:
import pandas as pd
import glob
import os
import numpy as np
import filepath
import send_email 
import pygsheets
import infrastructure 
import time
from datetime import datetime, timedelta
import filepath 
localfolder = filepath.output_folder 

In [2]:
start_time = time.time()
current_date = datetime.now()
two_months_ago = current_date - timedelta(days=60)

In [3]:
# sms_new = pd.read_csv(filepath.output_folder+'SS_LC_merged_data_new.csv')
# #sms_new['Revenue'].sum()
# revenue_sum = sms_new[sms_new['Date'].isna()]['Revenue'].sum()
# revenue_sum

In [4]:
merged_data = pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
twomonthsago_new = merged_data[(merged_data['Date'].isna()) | (merged_data['Date'] <= two_months_ago)]
twomonthsago_new.to_csv(localfolder + 'SS_LC_merged_data_twomonthsago.csv', index =False)

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\2355827091.py:1: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')


In [5]:
##start

In [6]:
#import publisher from google sheet 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
rxmgref = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
publisher_raw  = rxmgref.worksheet('title','Publisher Configurations').get_as_df()
publisher_raw = publisher_raw.drop_duplicates(subset='PUBID', keep='last')
publisher_raw.to_csv(localfolder+'smartsheet/Publisher.csv')
publisher = publisher_raw[['DP.DS or DP.sV','PUBID']].drop_duplicates()

# import SMS OMS
offer_sheet = infrastructure.get_smartsheet('offers_sms')

## import La Nina. 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_sheet  = lanina.worksheet('title','La Nina (Current)').get_as_df()
lanina_sheet.to_csv(localfolder+'smartsheet/La_Nina.csv')

## EMIT 
emit = infrastructure.get_smartsheet('emit')
email_pubid = emit['Revenue Pub ID'].astype(str).str.split('.',expand = True)[0].unique().tolist()

jobs = pd.read_csv(localfolder + 'SMS_SC_SS_Jobs.csv', dtype={'job_id' :'Int64'})
jobs['start_tstamp'] = pd.to_datetime(jobs['start_tstamp'])
jobs = jobs[jobs['start_tstamp'] > two_months_ago]

creative_stats = pd.read_csv(localfolder + 'SMS_SC_SS_CreativesStats.csv', dtype={'CreativeId' :'str'})
creative_stats['Tstamp'] = pd.to_datetime(creative_stats['Tstamp'])
creative_stats['Tstamp'] = creative_stats['Tstamp'].dt.tz_localize(None)
creative_stats = creative_stats[creative_stats['Tstamp'] > two_months_ago]

raw_creative_stats = creative_stats
offers = pd.read_csv(localfolder + 'SMS_SC_SS_Offers.csv', usecols = ['name','hitpath_offer_id', 'type','status','redirect_type','conversion_event','conversion_payout','currency'] )
#historic_data = pd.read_csv(localfolder + 'SS_data.csv')#  data before Nov1

flows = pd.read_csv(localfolder + 'SMS_SC_SS_Flows.csv')
flows['Period'] = pd.to_datetime(flows['Period'].str[:10],format='mixed')
flows = flows[flows['Period'] > two_months_ago]

# add some information in flows
flows.loc[flows['Name'] =='SM.SRV.DSS-PR-SFW', 'Name'] = 'SM.SRV-DSS-PR-SFW'
flows['Revenue Source'] = 'Short Code - SS Flow'
flows['Code_Type'] = 'Short Code' 
flows['Shortcode Name'] = flows['Name'].str.split('-',expand = True)[1]
flows['Shortcode1'] = flows['Shortcode'].str.extract('(\d{5})')
flows['Offer'] = flows['OfferName'].str.extract(r'^(.*?)(?:\s*\(|\s*\[|$)')
flows = flows.drop('Shortcode',axis = 1)
flows['Dataset'] = flows['Name'].str.split('-',expand = True)[0]
flows['Dataset'] = flows['Dataset'].str.replace('JM.ONP','JET.ONP')
flows['Dataset'] = flows['Dataset'].str.replace('JM.NTC','JET.NTC')
flows.loc[flows['Name'].str.contains('DUP', na = False), 'Dataset'] =  flows['Name'].str.split('-',expand = True)[1]
#flows.loc[flows['Name'].str.contains('DUP', na = False), 'Shortcode Name'] =  flows['Name'].str.split('-',expand = True)[1].str.replace("I.","")
flows.loc[flows['Name'].str.contains('I.A4F', na = False), 'Dataset'] = 'I.A4F'
flows.loc[flows['Name'].str.contains('I.MFA', na = False), 'Dataset'] = 'I.MFA'
##added
flows.loc[flows['Name'].str.contains('I.N3G', na = False), 'Dataset'] = 'I.N3G'

flows.loc[flows['CountUnsub']==0,'CountUnsub'] = flows['CountAutoresponderStop']
#flows.loc[flows['CountDeliver']==0, 'CountDeliver'] = flows['CountSent']
#Add send stategy
flows['Send Strategy'] = np.where(
    flows['OfferName'].str.contains('PR|HE|SE'),'P','AR')

flows = flows.rename(columns=({'CountSent':'Delivered', 'CostDeliver':'Cost','CountClick':'Clicks','Shortcode1':'Shortcode', 'CountUnsub':'Optout'}))
flows['Date'] = flows['Period']
#flows['Date'] = pd.to_datetime(flows['Period'].str[:10],format='mixed')
flows['AR Flow ID']=flows['Id'].astype('str')
flows['AR Day'] = flows['OfferName'].str.extract('(\d+)\s*\(')
flows.loc[flows['AR Day'].isna(),'AR Day'] =  flows['OfferName'].str.extract('AR(\d)')[0]
flows.loc[flows['AR Day'].isna(),'AR Day'] = 'Null'
flows['Hitpath ID'] = flows['OfferName'].str.extract(r'(\d{4,5})')
flows.loc[flows['OfferName'].str.contains( 'OW',na = False), 'Hitpath ID'] =flows['OfferName'].str.split("_|[| ]",expand= True)[3]
flows['AR Flow'] = flows['AR Flow ID'] + '_Day_' + flows['Shortcode Name']+'_'+  flows['AR Day']
flows_clean = flows[['Hitpath ID','Date','Dataset','Offer','Shortcode','Shortcode Name','Revenue Source','Code_Type','Delivered','Optout','Cost','Clicks','AR Flow','AR Day','AR Flow ID','Send Strategy']]
flows_clean = flows_clean.merge(publisher[['DP.DS or DP.sV','PUBID']], left_on ='Dataset', right_on = 'DP.DS or DP.sV', how = 'left' )
flows_clean = flows_clean.rename(columns=({'PUBID':'Affiliate_id'}))
flows_clean.loc[flows_clean['Shortcode Name'] == 'FLC', 'Shortcode'] = '51797'
flows_clean.loc[flows_clean['Shortcode Name'] == 'CSS', 'Shortcode'] = '70610'
flows_clean.loc[flows_clean['Shortcode Name'] == 'HZB', 'Shortcode'] = '44345'
flows_clean.loc[flows_clean['Shortcode Name'] == 'MBC', 'Shortcode'] = '80837'
flows_clean.loc[flows_clean['Shortcode Name'] == 'DSS', 'Shortcode'] = '31232'
flows_clean.loc[flows_clean['Shortcode Name'] == 'SVT', 'Shortcode'] = '61659'
flows_clean.loc[flows_clean['Shortcode Name'] == 'UAA', 'Shortcode'] = '79743'
flows_clean.loc[flows_clean['Shortcode Name'] == 'PRC', 'Shortcode'] = '18333164159'
flows_clean.loc[flows_clean['Shortcode Name'] == 'UAATF', 'Shortcode'] = '18333641722'
flows_clean.loc[flows_clean['Shortcode Name'].str.contains('A4F'), 'Shortcode'] = '13022952715'
flows_clean.loc[flows_clean['Shortcode Name'].str.contains('MFA'), 'Shortcode'] = '18332686782'
flows_clean.loc[flows_clean['Shortcode Name'].str.contains('N3G'), 'Shortcode'] = '18332685688'
flows_clean.loc[flows_clean['Shortcode Name'].str.contains('FHB'), 'Shortcode'] = '18338987652'
flows_clean.loc[flows_clean['Shortcode Name'].str.contains('THM'), 'Shortcode'] = '18336145644'
flows_clean.loc[flows_clean['Shortcode Name'].str.contains('FRH'), 'Shortcode'] = '18333275117'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '51797',na = False), 'Shortcode Name'] = 'FLC'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '70610',na = False), 'Shortcode Name'] = 'CSS'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '44345',na = False), 'Shortcode Name'] = 'HZB'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '80837',na = False), 'Shortcode Name'] = 'MBC'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '31232',na = False), 'Shortcode Name'] = 'DSS'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '61659',na = False), 'Shortcode Name'] = 'SVT'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '79743',na = False), 'Shortcode Name'] = 'UAA'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18333164159',na = False), 'Shortcode Name'] = 'PRC'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18333641722',na = False), 'Shortcode Name'] = 'UAATF'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '13022952715',na = False), 'Shortcode Name'] = 'A4FLC'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '15612023538',na = False), 'Shortcode Name'] = 'A4FLC'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18332686782',na = False), 'Shortcode Name'] = 'MFATF'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18332685688',na = False), 'Shortcode Name'] = 'N3GTF'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18338987652',na = False), 'Shortcode Name'] = 'FBHTF'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18336145644',na = False), 'Shortcode Name'] = 'THMTF'
flows_clean.loc[flows_clean['Shortcode'].str.contains( '18333275117',na = False), 'Shortcode Name'] = 'FRHTF'
# change some segment name 
creative_stats['Segment'] = creative_stats['Segment'].str.replace('WW.YFA','WWM.YFA')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('ARM.CR','AI.CC')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('RH.3CS','RHD.CC')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('ED.247L','EDM.247L')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('PA.SWP','PA.PS')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('FM.YS','FSM.YS')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('CM_','CM.OSR_')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('CM.OSR.OSR','CM.OSR')
creative_stats['Segment'] = creative_stats['Segment'].str.replace('CM.OSR.OSR.OSR','CM.OSR')
creative_stats = creative_stats.rename(columns=({'DeliverCount':'Delivered', 'TotalCost':'Cost','ClickCount':'Clicks','UnsubCount':'Optout'}))
creative_id_na = creative_stats.loc[creative_stats['CreativeId'].isna()]
jobs_optout = jobs[['id','optout']]
creative_stats =  creative_stats.merge(jobs_optout, left_on = 'JobId' ,right_on='id', how='left')
jobid_count = creative_stats['JobId'].value_counts().to_dict()
creative_stats['Optout'] = creative_stats.apply(lambda row: row['optout'] / jobid_count[row['JobId']], axis=1)
creative_stats = creative_stats.groupby(['JobId', 'Tstamp', 'Offer', 'Segment', 'CreativeId', 'CreativeName','Creative']).sum().reset_index()
print(creative_stats['Delivered'].sum())
jobs['segments'] = jobs['segments'].str.replace('WW.YFA','WWM.YFA')
jobs['segments'] = jobs['segments'].str.replace('ARM.CR','AI.CC')
jobs['segments'] = jobs['segments'].str.replace('RH.3CS','RHD.CC')
jobs['segments'] = jobs['segments'].str.replace('ED.247L','EDM.247L')
jobs['segments'] = jobs['segments'].str.replace('PA.SWP','PA.PS')
jobs['segments'] = jobs['segments'].str.replace('FM.YS','FSM.YS')
jobs['segments'] = jobs['segments'].str.replace('CM_','CM.OSR_')
jobs['segments'] = jobs['segments'].str.replace('CM.OSR.OSR','CM.OSR')
jobs['segments'] = jobs['segments'].str.replace('CM.OSR.OSR.OSR','CM.OSR')


#clean jobs file
#drop columns with nulls
jobs.isna().sum()
drop_columns_jobs = ['remote_status']
jobs.drop(columns=drop_columns_jobs, inplace=True, axis=1)
jobs = jobs.rename(columns={ 'id':'job_id', 'name':'job_name'})
#convert date fields to correct format
jobs['start_tstamp'] = pd.to_datetime(jobs['start_tstamp'])
jobs['end_tstamp'] = pd.to_datetime(jobs['end_tstamp'])
jobs['scheduled_tstamp'] = pd.to_datetime(jobs['scheduled_tstamp'],format = 'mixed')
jobs['Scheduling Time'] = jobs['scheduled_tstamp'].dt.tz_convert('US/Pacific').dt.strftime('%Y-%m-%d %H:%M')
creative_stats =  creative_stats.merge(jobs[['job_id','Scheduling Time']], left_on = 'JobId' ,right_on='job_id', how='left')
creative_id_na =  creative_id_na.merge(jobs[['job_id','Scheduling Time']], left_on = 'JobId' ,right_on='job_id', how='left')

#clean creative_stats
creative_stats.isna().sum()
creative_stats['Tstamp'] = pd.to_datetime(creative_stats['Tstamp'])

#clean offers
offers.isna().sum()
offers = offers.rename(columns={ 'id':'offer_id', 'name':'offer'})

#####engagement stats for all sends#####
#merge above df with reveneu file to get all of reveneu information
## combine all months revenue CSVs into one master-revenue file.
os.chdir(localfolder + "SMS Rev")

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f, dtype={'advertiser_name':'str','campaign_name':'str'}, low_memory=False) for f in all_files])
#combined_csv = pd.concat([pd.read_csv(f) for f in all_files ])
####Exporting master revenue file####
combined_csv.to_csv( localfolder + "SMS_master_revenue.csv", index=False, encoding='utf-8-sig')

combined_csv = pd.read_csv(localfolder + 'SMS_master_revenue.csv', dtype={'advertiser_name':'str','campaign_name':'str'})
daily_revenue = combined_csv
daily_revenue['date'] = pd.to_datetime(daily_revenue['date'],format = 'mixed').dt.date
daily_revenue_gpby = daily_revenue.groupby(['affiliate_id','date'])['amount'].sum().reset_index()
jump_page_csv = pd.read_csv( localfolder+"jumppage_click.csv")
jumppage_clicks = jump_page_csv.rename(columns = {'subid_1':"subid_2"})
print(combined_csv['amount'].sum())
combined_csv = pd.concat([combined_csv,jumppage_clicks])
print(combined_csv['amount'].sum())
print(combined_csv['Jump Page Clicks'].sum())

#check subids
subs = ["subid_1","subid_2", "subid_5"]
def sid(d):
    sid = "subid_3"
    for j in subs:
        if (d[f"{j} uc"]>=5) & (d[f"{j} ucs"]==0):
            sid = j
    return d[sid]
for i in subs:
    combined_csv[i] = combined_csv[i].astype(str)
    combined_csv[f"{i} uc"] = combined_csv[i].str.count("_")
    combined_csv[f"{i} ucs"] = combined_csv[i].str.count(":")
#merge all subids from 1,2 and 5 in subid
combined_csv["sub_id"] = combined_csv.apply(sid,axis=1)
combined_csv['sub_id'] = combined_csv['sub_id'].str.replace(
    'SS_SC_HZB_44345_461871_0_13074_868123_',
    'SS_TF_MFA_18332686782_461871_AR0_13074_868123_',
    regex=False
)
########## SS BEGIN #########
combined_csv['first_split']=combined_csv['sub_id'].str.split('_').str[0]
""" 
jumppage_clicks["sub_id"] = jumppage_clicks["subid_1"]
subs = ["sub_id"]
# doing same cleaning with jumpapge files  
for i in subs:
    jumppage_clicks[i] = jumppage_clicks[i].astype(str)
    jumppage_clicks[f"{i} uc"] = jumppage_clicks[i].str.count("_")
#merge all subids from 1,2 and 5 in subid

jumppage_clicks['first_split']=jumppage_clicks['sub_id'].str.split('_').str[0]
"""
# get the date based on subid date 
combined_csv = combined_csv.reset_index(drop=True)
combined_csv['Dash_Date_from_subid'] = combined_csv['sub_id'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
combined_csv['Dash_Date_from_subid'] = pd.to_datetime(combined_csv['Dash_Date_from_subid'], format='%d%b%y',errors='coerce')
combined_csv['date'] = pd.to_datetime(combined_csv['date'],errors='coerce')
combined_csv.loc[combined_csv['Dash_Date_from_subid'].isna(),'Dash_Date_from_subid'] = combined_csv['date']
combined_csv['date'] = combined_csv['Dash_Date_from_subid']
combined_csv = combined_csv[combined_csv['date'] > two_months_ago]
combined_csv.loc[combined_csv['campaign_id']==8202]
combined_csv.loc[(combined_csv['affiliate_id'] == 460001) & (combined_csv['site_id'].isna()== False),'affiliate_id'] = combined_csv['site_id']
combined_csv['affiliate_id'] = combined_csv['affiliate_id'].astype('Int64')

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


15029726


D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\845277884.py:170: DtypeWarning: Columns (7,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_csv = pd.read_csv(localfolder + 'SMS_master_revenue.csv', dtype={'advertiser_name':'str','campaign_name':'str'})


7426282.401051002
7426282.401051002
15854293.0


## Long Code

In [7]:
# Import Long Code data from MD

# lc = gc.open_by_url('https://docs.google.com/spreadsheets/d/1De9UzMw5POUuS90s7dMKsYNTQVnxTj1dvAvwMhOCC5U/edit#gid=0') 
# lc_wk_df  = lc.worksheet('title','Long Code').get_as_df()
# lc_wk_df['SMS Cost'] = lc_wk_df['SMS Cost'].str.split('$',expand = True)[1].astype('float')
# # import MD_
# md_optout = pd.read_csv(localfolder + 'MD_optout.csv', index_col= False)
# md_optout[['campaign','Date']]=md_optout['campaign'].str.split('\nStarted At: ',expand = True)
# md_optout['Date'] = pd.to_datetime(md_optout['Date'], format='%Y-%m-%d %H:%M:%S',errors='coerce')
# md_optout['campaign'] = md_optout['campaign'].str.extract('([a-zA-Z].*)', expand=True)

# lc_wk_df['Date'] = pd.to_datetime(lc_wk_df['Date'], errors='coerce')
# lc_wk_df = lc_wk_df[lc_wk_df['Date'] > two_months_ago]
# lc_wk_df = lc_wk_df.merge(md_optout[['Date','campaign','optout']], left_on = ['Offer','Date'], right_on = ['campaign','Date'], how = 'left')
# # rename to Optout
# lc_wk_df = lc_wk_df.rename(columns = {'Offer':'Sub_Id','optout':'Optout'})
# lc_rev = combined_csv.loc[(combined_csv['first_split']=='MD') | (combined_csv['first_split']=='TB') ]
# #lc_rev = combined_csv.loc[(combined_csv['first_split']=='MD') ]
# lc_rev.loc[lc_rev['Dash_Date_from_subid'].isna(), 'Dash_Date_from_subid'] = lc_rev['date']
# lc_rev =lc_rev.rename(columns = {"Dash_Date_from_subid":'Date','sub_id':'Sub_Id'})
# #  define the pattern
# #pattern = r'(.+\d{1,2}[A-Za-z]{3}\d{2})'
# # extract the pattern
# #lc_rev['Sub_Id'] = lc_rev['sub_id'].str.extract(pattern)
# lc_rev_summary = lc_rev.groupby(['Sub_Id','Date'])['amount'].sum().reset_index()
# lc_df = pd.concat([lc_wk_df,lc_rev_summary],axis = 0, ignore_index=True)
# # remove Date from lc_df
# lc_df['Date']= pd.to_datetime(lc_df['Date'],errors='coerce')
# lc_df = lc_df.groupby(['Sub_Id','Date']).sum().reset_index()  
# lc_df = lc_df.fillna(0)
                           
                           
# lc_df.loc[lc_df['Sub_Id'] == '01JUN23_8838_W4_EDU_SVT','Sub_Id'] = 'MD_LC_OMG_SVT_460918__8838_OT_01JUN23'
# lc_df.loc[lc_df['Sub_Id'] == '05Jun23_MD_LC_OMG_SVT_460918_W1_11714_T1','Sub_Id'] = 'MD_LC_OMG_SVT_460918__11714_OT_05Jun23'
# lc_df.loc[lc_df['Sub_Id'] == '11714-2_OMG_SVT','Sub_Id'] = 'MD_LC_OMG_SVT_460918__11714_OT_'
# lc_df.loc[lc_df['Sub_Id'] == 'MD_LC_OMG_SVT_460919_00066_9088__29Jul23','Sub_Id'] = 'MD_LC_OMG_SVT_460919_00066_9088_P_29Jul23'
# lc_df.loc[lc_df['Sub_Id'] == 'MD_LC_OMG_SVT_460920_00068_12076__30Jul23','Sub_Id']  = 'MD_LC_OMG_SVT_460920_00068_12076_P_30Jul23'
# lc_df['split_column'] = lc_df['Sub_Id'].str.split('_')
# lc_df['subid_uc'] = lc_df['Sub_Id'].str.count('_')
# #lc_df['Date'] = lc_df.loc[lc_df['Date'].isna(), 'Date'] = pd.to_datetime(lc_df['split_column'].str[8], format='%d%b%y',errors='coerce')
# lc_df['Send Strategy'] = lc_df['split_column'].str[7]
# lc_df['Hitpath_Offer_ID'] = lc_df['split_column'].str[6]
# # detect a 6digit number that start with "46" from Sub_Id
# lc_df['Affiliate_Id'] = lc_df['Sub_Id'].str.extract(r'(46\d{4})')
# #lc_df['Affiliate_Id'] = lc_df['split_column'].str[4]
# lc_df['Long Code Content ID'] = lc_df['split_column'].str[5]
# lc_df['Shortcode Name'] = ''
# lc_df.loc[lc_df['split_column'].str[3] == 'SVT','Shortcode Name' ] = 'SVTLC'
# lc_df.loc[lc_df['split_column'].str[3] == 'UAA','Shortcode Name' ] = 'UAALC'
# lc_df['Send Strategy'] = lc_df['Send Strategy'].str.replace('T0','OT')
# lc_df['Send Strategy'] = lc_df['Send Strategy'].str.replace('T1','OT') 
# lc_df.loc[lc_df['split_column'].str[0] == 'TB', 'Revenue Source'] = 'Long Code - Textback'
# lc_df.loc[lc_df['split_column'].str[0] == 'MD', 'Revenue Source'] = 'Long Code - Mobile Drips'
# lc_df.loc[lc_df['split_column'].str[0] == 'TB', 'Send Strategy'] = 'P'
# lc_df.loc[lc_df['split_column'].str[0] == 'TB', 'Long Code Content ID'] = np.nan

# # currently we don't consider split test content option 
# lc_df['amount'] = lc_df['amount'].fillna(0)
# print(lc_df.loc[lc_df['split_column'].str[0] == 'MD',]['amount'].sum())
# lc_lanina = lanina_sheet[(lanina_sheet['Long Code Content ID'].isna() == False) & ((lanina_sheet['Long Code Content ID'] !=""))]
# lc_lanina['Long Code Content ID'] = lc_lanina['Long Code Content ID'].astype(str).str.zfill(5)
# lc_df_full = lc_df.merge(lc_lanina[['Long Code Content ID','Reporting Content ID','Content']],copy = False, how= 'left', on = 'Long Code Content ID')
# print(lc_df_full.loc[lc_df_full['split_column'].str[0] == 'MD',]['amount'].sum())
# lc_df_full['Code_Type'] = 'Long Code'
# lc_df_full_11 = lc_df_full[lc_df_full['Date'] >= '2022-11-01']


# lc_df_full = lc_df_full.rename(columns = {'Qty':'Sent','Daily Success Qty':'Delivered','Fail Qty': 'Undelivered',\
#                             'Clicks Qty':'Clicks','SMS Cost':'Cost',\
#                             'amount':'Revenue','Long Code Content ID':'Creative_Id', 'Reporting Content ID':'Creativename','Content':'Creative'})

# lc_df_full = lc_df_full[['Date','Affiliate_Id', 'Hitpath_Offer_ID','Sent','Delivered','Undelivered','Clicks','Optout',\
#        'Cost','Revenue', 'Send Strategy', 'Shortcode Name', 'Revenue Source', 'Code_Type','Sub_Id','Creative_Id','Creativename','Creative']]

## Short Code

In [8]:
# combined_csv['date'].unique()

In [9]:
combined_csv['date'] = pd.to_datetime(combined_csv['date'])
combined_csv_ss_only = combined_csv[(combined_csv['first_split']=='SS')  |  (combined_csv['first_split']=='SMS')] 
combined_csv_ss_only = combined_csv_ss_only[combined_csv_ss_only['date']>='2022-11-01'].reset_index(drop=True)

combined_csv_ss_only['subid_uc'] = combined_csv_ss_only.sub_id.str.count('_')
#combined_csv_ss_only['subid_uc']=combined_csv.sub_id.str.count('_')

#ignoring the subids that  are not formatted correctly ex: SS_HZB_{{datasource_id}}_{{job_id}}_ALL.SMS_10434_SM_44345_{{today_d}}{{today_mon}}{{today_yy}}_1_{{member_id}}
#they have very less revenue under them
#these are with sub id uc <12
combined_csv_ss_only = combined_csv_ss_only[combined_csv_ss_only['subid_uc'] < 12]

combined_csv_ss_only.loc[combined_csv_ss_only['advertiser_name'] == 'NIC','campaign_id'] = 10267
Offer_name = combined_csv_ss_only.groupby(['campaign_id','campaign_name']).count().reset_index()[['campaign_id','campaign_name']]

# get jump page sum and revenue sum 
combined_csv_ss_only['split_column'] = combined_csv_ss_only['sub_id'].str.split('_')
combined_csv_ss_only.loc[(combined_csv_ss_only['first_split'] == 'SS')  , 'offer_id'] = combined_csv_ss_only['split_column'].str[6]
combined_csv_ss_only.loc[(combined_csv_ss_only['first_split'] == 'SMS')  , 'offer_id'] = combined_csv_ss_only['split_column'].str[7]
combined_csv_ss_only.loc[combined_csv_ss_only['offer_id'].str.contains("^(\d{4,5}).*|OW", regex = True, na = False) == False , 'offer_id'] =  combined_csv_ss_only['split_column'].str[5]
combined_csv_ss_only.loc[combined_csv_ss_only['offer_id'].str.contains("^(\d{4,5}).*|OW", regex = True, na = False) == False, 'offer_id'] =  combined_csv_ss_only['split_column'].str[4]

combined_csv_ss_only['campaign_id']= combined_csv_ss_only['campaign_id'].astype('str').str.split('.',expand = True)[0]

combined_csv_ss_only['Jump Page Version'] = '0'
combined_csv_ss_only.loc[combined_csv_ss_only['offer_id'].str.contains('v',na = False),'Jump Page Version'] = combined_csv_ss_only['offer_id'].str.split('v',expand = True)[1]


#combined_csv_ss_only['c1'] = combined_csv_ss_only['sub_id'].str[:-9]

# import offer wall database
ofwall_df = infrastructure.get_smartsheet('ow_sms')
ofwall_df = ofwall_df.rename(columns = {'Hitpath Offer ID':'Offer Wall ID'})
offer_wall_id = ofwall_df['Offer Wall ID'].astype(str).unique().tolist()
combined_csv_ss_only['Offer Type'] = 'Single Offer'
combined_csv_ss_only.loc[combined_csv_ss_only['offer_id'].isin(offer_wall_id), 'Offer Type'] = 'Offer Wall'
offerwall = combined_csv_ss_only.loc[(combined_csv_ss_only['Offer Type']=='Offer Wall') |((combined_csv_ss_only['offer_id']!= combined_csv_ss_only['campaign_id'])& (combined_csv_ss_only['campaign_id'].isna() == False) &  (combined_csv_ss_only['campaign_id'] != 'nan') & (combined_csv_ss_only['offer_id'].str.contains("v",na = True) == False))  ,]

combined_csv_ss_only['Offer Wall Id'] = ''
combined_csv_ss_only.loc[(combined_csv_ss_only['Offer Type']=='Offer Wall')  , 'Offer Wall Id'] = combined_csv_ss_only['offer_id']
#combined_csv_ss_only['offer_id'] = combined_csv_ss_only['offer_id'].astype('str').str.extract(r'\b(\d{4,5}).*')
#combined_csv_ss_only.loc[combined_csv_ss_only['Offer Type']=='Offer Wall' , 'campaign_id'] = combined_csv_ss_only['offer_id'].str.split('OW',expand = True)[0]
combined_csv_ss_only.loc[(combined_csv_ss_only['Offer Type']=='Offer Wall')  , 'campaign_id'] = combined_csv_ss_only['offer_id']
combined_csv_ss_only.loc[(combined_csv_ss_only['campaign_id'].isna()) | (combined_csv_ss_only['campaign_id'] == 'nan'),'campaign_id' ] = combined_csv_ss_only['offer_id']
combined_csv_ss_only.loc[(combined_csv_ss_only['campaign_id'] != combined_csv_ss_only['offer_id'])  ,'campaign_id'] = combined_csv_ss_only['offer_id']
# rename offer id to offer wall id


combined_csv_ss_only = combined_csv_ss_only[['date', 'campaign_id', 'Offer Wall Id','Jump Page Version',
       'affiliate_id',  'amount', 'Jump Page Clicks',
        'sub_id', 'first_split','Dash_Date_from_subid', 'subid_uc','Offer Type']]

#combined_csv_ss_only['amount'] = combined_csv_ss_only['amount'].fillna(0)
combined_csv_ss_only[ 'Jump Page Clicks'] = combined_csv_ss_only[ 'Jump Page Clicks'].fillna(0)
#combined_csv_ss_only = combined_csv_ss_only.groupby(['date', 'campaign_id', 'affiliate_id', 'Jump Page Version','Offer Wall Id', 'sub_id', 'first_split','Dash_Date_from_subid', 'subid_uc','Offer Type'])[[ 'amount', 'Jump Page Clicks']].sum().reset_index()

combined_csv_ss_only = combined_csv_ss_only.groupby([
    'date', 'campaign_id', 'affiliate_id', 'Jump Page Version',
    'Offer Wall Id', 'sub_id', 'first_split', 'Dash_Date_from_subid',
    'subid_uc', 'Offer Type'
]).agg({
    'amount': ['sum', 'count'],  # Summing and counting for the 'amount' column
    'Jump Page Clicks': 'sum'     # Summing for the 'Jump Page Clicks' column
}).reset_index()
 
combined_csv_ss_only.columns = [
    'date', 'campaign_id', 'affiliate_id', 'Jump Page Version',
    'Offer Wall Id', 'sub_id', 'first_split', 'Dash_Date_from_subid',
    'subid_uc', 'Offer Type', 'amount','conversions','Jump Page Clicks'
]

# use for verification 
jumppageclicks1 = combined_csv_ss_only['Jump Page Clicks'].sum()
revenue1 = combined_csv_ss_only['amount'].sum()



D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\1824205372.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  combined_csv_ss_only.loc[combined_csv_ss_only['offer_id'].str.contains("^(\d{4,5}).*|OW", regex = True, na = False) == False , 'offer_id'] =  combined_csv_ss_only['split_column'].str[5]
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\1824205372.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  combined_csv_ss_only.loc[combined_csv_ss_only['offer_id'].str.contains("^(\d{4,5}).*|OW", regex = True, na = False) == False, 'offer_id'] =  combined_csv_ss_only['split_column'].str[4]


In [10]:
#*** Extracting creative information. ****#
# find creative id from list of creative ids from creativestats file.
combined_csv_ss_only['split_column'] = combined_csv_ss_only['sub_id'].str.split('_')
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SS'].index , 'creative_id'] = combined_csv_ss_only['split_column'].str[7]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SMS'].index , 'creative_id'] = combined_csv_ss_only['split_column'].str[8]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SS'].index , 'Job_id'] = combined_csv_ss_only['split_column'].str[5]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SMS'].index , 'Job_id'] = combined_csv_ss_only['split_column'].str[6]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SS'].index , 'Code_Type'] = combined_csv_ss_only['split_column'].str[1]
combined_csv_ss_only.loc[combined_csv_ss_only[combined_csv_ss_only['first_split'] == 'SMS'].index , 'Code_Type'] = combined_csv_ss_only['split_column'].str[2] 

# 
combined_csv_ss_only.loc[combined_csv_ss_only['Code_Type'] == 'TF', 'Code_Type' ] ='Toll Free' 
combined_csv_ss_only.loc[combined_csv_ss_only['Code_Type'] == 'SC', 'Code_Type' ] ='Short Code' 
combined_csv_ss_only.loc[combined_csv_ss_only['Code_Type'] == 'FLC', 'Code_Type' ] ='Short Code'
combined_csv_ss_only.loc[combined_csv_ss_only['Code_Type'] == 'LC', 'Code_Type' ] ='Long Code'
# some AR subids in december formatted incorrectly, some residuals formatted incorrectly.
#creative ids are length>6, creative ids len =1 are ARs, ignoring both the cases and repalcing creative ids with nans for rest of the length values
combined_csv_ss_only['creative_idlen'] = combined_csv_ss_only['creative_id'].str.len()
combined_csv_ss_only.loc[combined_csv_ss_only[~combined_csv_ss_only['creative_idlen'].isin([1,6])].index,'creative_id']=np.nan 

# identify shortcode 
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('FLC', na=False), 'Shortcode Name'] = 'FLC'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('CSS', na=False), 'Shortcode Name'] = 'CSS'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('HZB', na=False), 'Shortcode Name'] = 'HZB'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('MBC', na=False), 'Shortcode Name'] = 'MBC'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('DSS', na=False), 'Shortcode Name'] = 'DSS'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('SVT', na=False), 'Shortcode Name'] = 'SVT'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('UAA', na=False), 'Shortcode Name'] = 'UAA'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('PRC', na=False), 'Shortcode Name'] = 'PRC'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('UAATF', na=False), 'Shortcode Name'] = 'UAATF'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('MFA', na=False), 'Shortcode Name'] = 'MFATF'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('A4F', na=False), 'Shortcode Name'] = 'A4FLC'
#added
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('N3G', na=False), 'Shortcode Name'] = 'N3GTF'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('FBH', na=False), 'Shortcode Name'] = 'FBHTF'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('THM', na=False), 'Shortcode Name'] = 'THMTF'
combined_csv_ss_only.loc[combined_csv_ss_only['sub_id'].str.contains('FRH', na=False), 'Shortcode Name'] = 'FRHTF'
combined_csv_ss_only.loc[(combined_csv_ss_only['Shortcode Name'] == 'SVT') & (combined_csv_ss_only['Code_Type'] == 'LC' ) , 'Shortcode Name'] = 'SVTLC'
combined_csv_ss_only.loc[(combined_csv_ss_only['Shortcode Name'] == 'UAA') & (combined_csv_ss_only['Code_Type'] == 'LC' ) , 'Shortcode Name'] = 'UAALC'

combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'FLC', 'Shortcode'] = '51797'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'CSS', 'Shortcode'] = '70610'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'HZB', 'Shortcode'] = '44345'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'MBC', 'Shortcode'] = '80837'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'DSS', 'Shortcode'] = '31232'
combined_csv_ss_only.loc[(combined_csv_ss_only['Shortcode Name'] == 'SVT') , 'Shortcode'] = '61659'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'UAA', 'Shortcode'] = '79743'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'PRC', 'Shortcode'] = '18333164159'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'UAATF', 'Shortcode'] = '18333641722'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'MFATF', 'Shortcode'] = '18332686782'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'A4FLC', 'Shortcode'] = '13022952715'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'THMTF', 'Shortcode'] = '18336145644'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'FRHTF', 'Shortcode'] = '18333275117'
#added
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'N3GTF', 'Shortcode'] = '18332685688'
combined_csv_ss_only.loc[combined_csv_ss_only['Shortcode Name'] == 'FBHTF', 'Shortcode'] = '18338987652'

#reformat jobs from residuals and AR
combined_csv_ss_only['job_idlen'] = combined_csv_ss_only['Job_id'].str.len()
combined_csv_ss_only.loc[combined_csv_ss_only[~combined_csv_ss_only['job_idlen'].isin([1,6])].index,'Job_id']=np.nan 
combined_csv_ss_only_reformatjobs = combined_csv_ss_only[combined_csv_ss_only['Job_id'].isna() ]
jobs['job_id'] =jobs['job_id'].astype('str')
"""
for i in jobs['job_id']:
    check_len = combined_csv_ss_only_reformatjobs[combined_csv_ss_only_reformatjobs['sub_id'].str.contains(i)]
    if len(check_len) > 0:
        combined_csv_ss_only_reformatjobs.loc[check_len.index,'Job_id'] = i
"""
# Extract the job IDs to a set for faster membership testing
creative_stats['JobId'] = creative_stats['JobId'].astype('str')
job_ids = set(jobs['job_id']) | set(creative_stats['JobId']) - set(['0'])
#creativeid = set(creative_stats['Creative_Id'])
#combined_csv_ss_only['creative_id'] = combined_csv_ss_only_reformatjobs['sub_id'].str.extract(f'({"|".join(creativeid)})')

# Use str.extract to extract the job ID from the sub_id column
combined_csv_ss_only_reformatjobs['Job_id'] = combined_csv_ss_only_reformatjobs['sub_id'].str.extract(f'({"|".join(job_ids)})')

combined_csv_ss_only =   pd.concat([combined_csv_ss_only_reformatjobs,(combined_csv_ss_only[~combined_csv_ss_only['Job_id'].isna()])] )

combined_csv_ss_only = combined_csv_ss_only[combined_csv_ss_only['creative_idlen']!= 8]
combined_csv_ss_only = combined_csv_ss_only[~combined_csv_ss_only['creative_idlen'].isna()]
#get all creatives information.
combined_csv_ss_only['Job_id'] = combined_csv_ss_only['Job_id'].astype('float')
combined_csv_ss_only['creative_id'] = combined_csv_ss_only['creative_id'].replace('6Oct22', np.nan)
combined_csv_ss_only['creative_id'] = combined_csv_ss_only['creative_id'].astype('float')

creative_stats['CreativeId'] = creative_stats['CreativeId'].astype('float')
# make sure we don't want to merge jobid = 0 
#combined_csv_ss_only = combined_csv_ss_only.groupby(['Job_id', 'creative_id'])['amount'].sum().reset_index()
#combined_csv_ss_creative = combined_csv_ss_only.merge(creative_stats, left_on = ['Job_id', 'creative_id'], right_on = ['JobId', 'CreativeId'], how = 'left')
combined_csv_ss_only['date']  = pd.to_datetime(combined_csv_ss_only['date'] )
combined_csv_ss_only['date'] = combined_csv_ss_only['date'].dt.date

print(combined_csv_ss_only['Jump Page Clicks'].sum())

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\3624828272.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_ss_only_reformatjobs['Job_id'] = combined_csv_ss_only_reformatjobs['sub_id'].str.extract(f'({"|".join(job_ids)})')


1689142.0


### SS flow, SS jobs and SS data without jobid(such as optin) 

In [11]:
combined_csv_ss_only['Hitpath ID'] = combined_csv_ss_only['campaign_id']
#combined_csv_ss_only['Dash_Date_from_subid'] = pd.to_datetime(combined_csv_ss_only['Dash_Date_from_subid'], format="%d%b%y")
# flow data (51797 & 80837 & jobid = 0)
#  by 2023/4/11, we don't include creative id. Subid has that info

flows_clean1 = flows_clean.groupby(['Hitpath ID', 'Date','Offer', 'Dataset', 'Shortcode', 'Shortcode Name',\
       'Revenue Source', 'Code_Type', 'AR Flow', 'AR Day', 'AR Flow ID', 'DP.DS or DP.sV','Affiliate_id','Send Strategy']).sum().reset_index()
flows_clean1.loc[flows_clean1['AR Day']=='Null', 'AR Day'] = 0 
flows_clean1['AR Day'] = flows_clean1['AR Day'].astype(int)
flows_clean1 = flows_clean1.groupby(['Hitpath ID', 'Date', 'Offer','Dataset', 'Shortcode', 'Shortcode Name',
       'Revenue Source', 'Code_Type',  'DP.DS or DP.sV',
       'Affiliate_id','Send Strategy']).sum().reset_index()
flows_clean1['AR Day'] = flows_clean1['AR Day'].astype(str)

combined_csv_ss_only.loc[(combined_csv_ss_only['split_column'].str[5].str.contains('AR',na = False)) & (combined_csv_ss_only['Hitpath ID'].isna()) ,'Hitpath ID'] =combined_csv_ss_only['split_column'].str[6].str.extract(r'\b(\d{4,5}).*')
combined_csv_ss_ar_flow = combined_csv_ss_only[(combined_csv_ss_only['sub_id'].str.contains('AR|PR|HE|SE')) |  (combined_csv_ss_only['Job_id']==0)]
combined_csv_ss_ar_flow['flow_abbr'] = 'FLOW'
combined_csv_ss_ar_flow.loc[(combined_csv_ss_ar_flow['sub_id'].str.contains('AR|PR|HE|SE')),'flow_abbr'] = combined_csv_ss_ar_flow['split_column'].str[5]
flows_clean1['flow_abbr'] = 'FLOW'
flows_clean1.loc[(flows_clean1['Offer'].str.contains('AR|PR|HE|SE')),'flow_abbr'] = flows_clean1['Offer'].str.split("_",expand = True)[2]
####add conversions
combined_csv_ss_ar_flow1 = combined_csv_ss_ar_flow.groupby(['affiliate_id','Hitpath ID','Shortcode','flow_abbr','Dash_Date_from_subid','Offer Type','Jump Page Version','Offer Wall Id'],dropna = False)[['amount','conversions','Jump Page Clicks']].sum().reset_index()
combined_csv_ss_ar_flow1 = combined_csv_ss_ar_flow1.rename(columns=({'amount': 'Revenue','affiliate_id':'affiliate_id1'}))
flows_clean1 = flows_clean1.groupby(['Hitpath ID', 'Date', 'Offer','flow_abbr','Dataset', 'Shortcode', 'Shortcode Name',\
       'Revenue Source', 'Code_Type', 'AR Day', \
       'DP.DS or DP.sV', 'Affiliate_id']).sum().reset_index()
print(combined_csv_ss_ar_flow1['Jump Page Clicks'].sum())
unique_id_count = combined_csv_ss_ar_flow1[['Dash_Date_from_subid','Hitpath ID','Shortcode','affiliate_id1']].value_counts().reset_index(name = 'count')
combined_csv_ss_ar_flow1 = combined_csv_ss_ar_flow1.merge(unique_id_count, on = ['Dash_Date_from_subid','Hitpath ID','Shortcode','affiliate_id1'],how = 'left')
flows_clean2 = flows_clean1.merge(combined_csv_ss_ar_flow1, left_on = ['Date','Hitpath ID','Shortcode','Affiliate_id','flow_abbr'],right_on = ['Dash_Date_from_subid','Hitpath ID','Shortcode','affiliate_id1','flow_abbr'],how = 'outer')
flows_clean2['count'] = flows_clean2['count'].fillna(1)
flows_clean2['Delivered'] = flows_clean2['Delivered'] /flows_clean2['count']
flows_clean2['Optout'] = flows_clean2['Optout'] /flows_clean2['count']
flows_clean2['Cost'] = flows_clean2['Cost'] /flows_clean2['count']
flows_clean2['Clicks'] = flows_clean2['Clicks'] /flows_clean2['count']
print(flows_clean2['Jump Page Clicks'].sum())

flows_clean2['Revenue Source'] = 'Short Code - SS Flow'
flows_clean2['Code_Type'] = 'Short Code'
flows_clean2.loc[flows_clean2['Date'].isna(), 'Date'] = flows_clean2['Dash_Date_from_subid']
flows_clean2.loc[(flows_clean2['Shortcode'] == '51797') & (flows_clean2['Shortcode Name'].isna()), 'Shortcode Name'] = 'FLC'
flows_clean2.loc[(flows_clean2['Shortcode'] == '80837') & (flows_clean2['Shortcode Name'].isna()), 'Shortcode Name'] = 'MBC'
flows_clean2['Revenue'] =  flows_clean2['Revenue'].fillna(0)
flows_clean2.columns = flows_clean2.columns.str.title()
#flows_clean2.loc[flows_clean2['Affiliate_Id'].isna(),'Affiliate_Id' ]  = flows_clean2['Affiliate_Id1']
flows_clean_no_delivered = flows_clean2.loc[flows_clean2['Affiliate_Id'].isna(), ] 
flows_clean_no_delivered['Affiliate_Id']= flows_clean_no_delivered['Affiliate_Id1']
flows_clean_no_delivered = flows_clean_no_delivered.dropna(axis =1 , how ='all') 
flows_clean2 = flows_clean2.loc[flows_clean2['Affiliate_Id'].isna() == False, ] 

check_no_match = combined_csv_ss_ar_flow.merge(flows_clean1, how = 'left', right_on = ['Date','Hitpath ID','Shortcode','Affiliate_id'],left_on = ['Dash_Date_from_subid','Hitpath ID','Shortcode','affiliate_id'])
check_no_match1 = check_no_match[check_no_match['Delivered'].isna()]
check_no_match_with_flow = check_no_match1.dropna(axis =1 , how ='all') 


#jobs without creative or job 
###Add 
combined_csv_ss_only1 = combined_csv_ss_only[~((combined_csv_ss_only['sub_id'].str.contains('AR|PR|HE|SE')) |  (combined_csv_ss_only['Job_id']==0))]
combined_csv_ss_only1 = pd.concat([combined_csv_ss_only1,check_no_match_with_flow]).reset_index(drop = True)
combined_csv_ss_creative_na = combined_csv_ss_only1[(combined_csv_ss_only1['creative_id'].isna())| (combined_csv_ss_only1['creative_id']<100000)| ( (combined_csv_ss_only1['Job_id'].isna()))]

#jobs with creative and job 
combined_csv_ss_creative_notna = combined_csv_ss_only1[~((combined_csv_ss_only1['creative_id'].isna()) | (combined_csv_ss_only1['creative_id']<100000)| ( (combined_csv_ss_only1['Job_id'].isna())))]

# get revenue and delivery stats for jobs with creatives
""" 
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.groupby(['Job_id', 'creative_id'])['amount'].sum().reset_index()
merge_frame = creative_stats[['JobId', 'CreativeId']].append(combined_csv_ss_creative_notna[['Job_id', 'creative_id']]).drop_duplicates()
creative_stats = creative_stats[creative_stats['Tstamp']>= '2022-11-01']
print(combined_csv_ss_creative_notna['amount'].sum())
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.merge(creative_stats, left_on = ['Job_id', 'creative_id'], right_on = ['JobId', 'CreativeId'], how = 'left')
print(combined_csv_ss_creative_notna['amount'].sum())
""" 
# get revenue and delivery stats for jobs with creatives

###add conversions
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.groupby(['Job_id', 'creative_id','Jump Page Version','Offer Wall Id','Offer Type'])[['amount','conversions','Jump Page Clicks']].sum().reset_index()
creative_stats11 = creative_stats[creative_stats['Tstamp']>= '2022-11-01']
creative_stats11 = creative_stats11.rename(columns=({'JobId': 'Job_id', 'CreativeId':'creative_id'}))
creative_stats11['Job_id'] = creative_stats11['Job_id'].astype('int')

merge_frame = pd.concat([creative_stats11[['Job_id', 'creative_id']],combined_csv_ss_creative_notna[['Job_id', 'creative_id']]]).drop_duplicates()
merge_frame['creative_id'] = merge_frame['creative_id'].astype('int')
creative_stats11['creative_id'] = creative_stats11['creative_id'].astype('int')
merge_frame = merge_frame.merge(combined_csv_ss_creative_notna, how = 'left')
merge_frame = merge_frame.merge(creative_stats11, how = 'left')

combined_csv_ss_creative_notna = merge_frame.fillna(0)

##add conversions
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna[['Offer','Job_id', 'creative_id','Jump Page Version','Offer Wall Id','Offer Type', 'CreativeName',\
       'Creative', 'Delivered', 'Cost', 'Optout', 'Clicks', 'amount','conversions','Jump Page Clicks']]
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.rename(columns=({'amount': 'Revenue'}))
#combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.drop_duplicates()
#combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.groupby(['date','Offer','Job_id','CreativeId', 'CreativeName', 'Creative', 'Delivered', 'Cost', 'Unsubcount', 'Clicks']).sum('Revenue').reset_index()

#get jobs information from jobs file
jobs['job_id'] =jobs['job_id'].astype('int')
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.merge(jobs[['job_id','job_name', 'shortcode', 'start_tstamp',
       'end_tstamp', 'scheduled_tstamp', 'status_text', 'segments','Scheduling Time']], left_on = 'Job_id', right_on = 'job_id', how = 'left')
combined_csv_ss_creative_notna['date'] = pd.to_datetime(combined_csv_ss_creative_notna['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.drop(columns ='job_id')
combined_csv_ss_creative_notna.columns = combined_csv_ss_creative_notna.columns.str.title()
# create send strategy 
combined_csv_ss_creative_notna['Send Strategy'] = np.nan 
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'P' in l),'Send Strategy'] = 'P'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'T' in l),'Send Strategy'] = 'OT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'OT' in l),'Send Strategy'] = 'OT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'PT' in l),'Send Strategy'] = 'PT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'AR' in l),'Send Strategy'] = 'AR'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'CT' in l),'Send Strategy'] = 'CT'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'MI' in l),'Send Strategy'] = 'MI'
combined_csv_ss_creative_notna.loc[ combined_csv_ss_creative_notna['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'JT' in l),'Send Strategy'] = 'JT'
# we didn't define the right send strategy for the following jobs in mamba
combined_csv_ss_creative_notna.loc[combined_csv_ss_creative_notna['Job_Name'].str.contains('SS_FLC_PN-FC-21DC-VZN_12305_P_01Nov23', na = False),'Send Strategy'] ='JT'
combined_csv_ss_creative_notna.loc[combined_csv_ss_creative_notna['Job_Name'].str.contains('SS_FLC_PN-FC-21DC-VZN_12305v1_CT_01Nov23', na = False),'Send Strategy'] ='JT'

combined_csv_ss_creative_notna = combined_csv_ss_creative_notna.reset_index(drop=True)

#get revenue stats for jobs without creatives.

##add conversions
combined_csv_ss_creative_na = combined_csv_ss_creative_na[['date','Job_id', 'campaign_id','amount','sub_id','conversions','Jump Page Clicks','Jump Page Version','Offer Wall Id','Offer Type']]
combined_csv_ss_creative_na = combined_csv_ss_creative_na.rename(columns=({'amount': 'Revenue'}))
#get delivery and click stats for jobs without creatives from job file
 
creative_id_na['Date'] = pd.to_datetime(creative_id_na['Tstamp']).dt.strftime("%Y-%m-%d")
creative_id_na11=creative_id_na[creative_id_na['Date'] >= '2022-11-01']
creative_id_na11['Campaign_Id'] = creative_id_na11['Offer'].str.split(' ',expand = True)[0]
creative_id_na11.loc[creative_id_na11['Campaign_Id'].str.isdigit() == False,'Campaign_Id' ]  = np.nan
creative_id_na11['Campaign_Id']  = creative_id_na11['Campaign_Id'].astype(float)
creative_id_na11['Shortcode Name'] = ''
creative_id_na11.loc[creative_id_na11['Offer'].str.contains('CSS', na = False),'Shortcode Name'] = 'CSS'
creative_id_na11.loc[creative_id_na11['Offer'].str.contains('HZB', na = False),'Shortcode Name'] = 'HZB'
# the jobid didn't use in the combined_csv_ss_creative_notna
#unjoined_creative_stats11 = creative_stats11[~creative_stats11['Job_id'].isin(combined_csv_ss_creative_notna['Job_Id'])]
creative_stats11['Date'] = pd.to_datetime(creative_stats11['Tstamp']).dt.strftime("%Y-%m-%d")
creative_stats11['Campaign_Id'] = creative_stats11['Offer'].str.split(' ',expand = True)[0]
creative_stats11.loc[creative_stats11['Campaign_Id'].str.isdigit() == False,'Campaign_Id' ]  = np.nan
creative_stats11['Campaign_Id']  = creative_stats11['Campaign_Id'].astype(float)
creative_stats11['Shortcode Name'] = ''
creative_stats11.loc[creative_stats11['Offer'].str.contains('CSS', na = False),'Shortcode Name'] = 'CSS'
creative_stats11.loc[creative_stats11['Offer'].str.contains('HZB', na = False),'Shortcode Name'] = 'HZB'

#
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('70610' ,na=False ),'shortcode']= '70610'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('44345',na=False),'shortcode']= '44345'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('70610' ,na=False ),'Shortcode Name']= 'CSS'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['sub_id'].str.contains('44345',na=False),'Shortcode Name']= 'HZB'
combined_csv_ss_creative_na['campaign_id'] = combined_csv_ss_creative_na['sub_id'].str.split('_',expand = True)[6]
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].str.isdigit()== False, 'campaign_id'] = combined_csv_ss_creative_na['sub_id'].str.split('_',expand = True)[5]
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].str.isdigit()== False, 'campaign_id'] = combined_csv_ss_creative_na['sub_id'].str.split('_',expand = True)[4]
combined_csv_ss_creative_na['affiliate_id'] = combined_csv_ss_creative_na['sub_id'].str.extract(r'(46\d{4})')
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].str.isdigit() == False,'campaign_id' ]  = np.nan
combined_csv_ss_creative_na['campaign_id'] = combined_csv_ss_creative_na['campaign_id'].astype(float)
#combined_csv_ss_creative_na['date'] = pd.to_datetime(combined_csv_ss_creative_na['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
print(jobs)
job_11 = jobs[jobs['scheduled_tstamp'] >= '2022-11-01']

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\2142074811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_ss_ar_flow['flow_abbr'] = 'FLOW'


713605.0
715028.0


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\2142074811.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flows_clean_no_delivered['Affiliate_Id']= flows_clean_no_delivered['Affiliate_Id1']


       job_id                                          job_name  \
35700  659072                   SS_MBC_I-RC-15DC_6322_P_27Apr24   
35701  659073              SS_FLC_I-RC-30DC-VZN_12440_P_27Apr24   
35702  659074                 SS_HZB_CM-OSR-30DC_6322_P_27Apr24   
35703  659075             SS_MBC_CM-OSR-30DC-TMO_6322_P_27Apr24   
35704  659076                SS_SVT_AMD-PL-30DC_11593_P_27Apr24   
...       ...                                               ...   
39230  678003             SS_DSS_GR-PL-30DC-TMO_12972_P_25Jun24   
39231  678004                 SS_FLC_GR-PL-30DC_12610_P_25Jun24   
39232  678005          SS_DSS_JET-ZTA-1PLD-21DC_11600_P_25Jun24   
39233  678431      SS_MBC_PN-FC-30PDC-OI-JUN23_13349_MI_25Jun24   
39234  678432  SS_FLC_PN-FC-30PDC-VZN-OI-JUN23_13113_MI_25Jun24   

                    offer  shortcode        start_tstamp          end_tstamp  \
35700  6322 - BB VAST MBC      80837 2024-04-27 12:30:40 2024-04-27 12:37:04   
35701   12440 - AO PJ FLC      5179

In [12]:
# print(flows_clean[(flows_clean['Date']>= '2023-10-01') &(flows_clean['Date']< '2023-11-01')]['Delivered'].sum())
# print(flows_clean1[(flows_clean1['Date']>= '2023-10-01') &(flows_clean1['Date']< '2023-11-01')]['Delivered'].sum())
# print(flows_clean2[(flows_clean2['Date']>= '2023-10-01') & (flows_clean2['Date']< '2023-11-01')]['Delivered'].sum())

In [13]:
combined_csv_ss_creative_na = combined_csv_ss_creative_na.merge(creative_id_na11[['Date','Scheduling Time','JobId','Offer','Campaign_Id', 'Segment', 'CreativeId', 'CreativeName','Shortcode Name',
       'Creative','Delivered',  'Optout', 'Clicks', 'Cost']], left_on = ['Job_id','campaign_id','date','Shortcode Name'], right_on = ['JobId','Campaign_Id','Date','Shortcode Name'], how = 'outer', copy = False)
combined_csv_ss_creative_na = combined_csv_ss_creative_na.merge(job_11[['job_id','job_name', 'offer', 'shortcode', 'scheduled_tstamp', 'status_text', 'segments', 
        'delivered', 'optout', 'clicks', 'cost']], left_on = ['Job_id'], right_on =['job_id'] , how = 'left' , copy = False)
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['date'].isna(), 'date'] = combined_csv_ss_creative_na['Date']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['campaign_id'].isna(), 'campaign_id'] = combined_csv_ss_creative_na['Campaign_Id']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Offer'].isna(), 'Offer'] = combined_csv_ss_creative_na['offer']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Delivered'].isna(), 'Delivered'] = combined_csv_ss_creative_na['delivered']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Optout'].isna(), 'Optout'] = combined_csv_ss_creative_na['optout']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Clicks'].isna(), 'Clicks'] = combined_csv_ss_creative_na['clicks']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Cost'].isna(), 'Cost'] = combined_csv_ss_creative_na['cost']
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Job_id'].isna(), 'Job_id'] = combined_csv_ss_creative_na['JobId']
combined_csv_ss_creative_na = combined_csv_ss_creative_na.drop(columns = ['job_id','Campaign_Id','Date','offer', 'delivered', 'optout', 'clicks', 'cost','JobId'])
#combined_csv_ss_creative_na = combined_csv_ss_creative_na.reset_index()
#combined_csv_ss_creative_na['date'] = pd.to_datetime(combined_csv_ss_creative_na['scheduled_tstamp']).dt.strftime("%Y-%m-%d")
combined_csv_ss_creative_na.columns = combined_csv_ss_creative_na.columns.str.title()

# create send strategy 
combined_csv_ss_creative_na['Send Strategy'] = np.nan 
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'P' in l),'Send Strategy'] = 'P'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'T' in l),'Send Strategy'] = 'OT'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'OT' in l),'Send Strategy'] = 'OT'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'PT' in l),'Send Strategy'] = 'PT'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'AR' in l),'Send Strategy'] = 'AR'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'CT' in l),'Send Strategy'] = 'CT'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'MI' in l),'Send Strategy'] = 'MI'
combined_csv_ss_creative_na.loc[ combined_csv_ss_creative_na['Job_Name'].str.split('_|[|]| ').str[-2:].astype('str').apply(lambda l: 'JT' in l),'Send Strategy'] = 'JT'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Job_Id'] == 0, 'Send Strategy'] = 'AR'
combined_csv_ss_creative_na.loc[combined_csv_ss_creative_na['Job_Id'] == 0, 'Revenue Source'] =  'Short Code - SS Jobs'
#combined_csv_ss_creative_na = combined_csv_ss_creative_na.reset_index(drop=True)

### Other Revenue

In [14]:
#case #3: revenue for email 
combined_csv_ss_exclude = combined_csv
combined_csv_ss_exclude['affiliate_id'] = combined_csv_ss_exclude['affiliate_id'].astype(str).str.split(".",expand = True)[0]
combined_csv_ss_exclude['site_id'] = combined_csv_ss_exclude['site_id'].astype(str).str.split(".",expand = True)[0]

combined_csv_ss_exclude =  combined_csv_ss_exclude.loc[(combined_csv_ss_exclude['affiliate_id'].isin(email_pubid)) | (combined_csv_ss_exclude['site_id'].isin(email_pubid)),] 
combined_csv_ss_exclude['date'] = pd.to_datetime(combined_csv_ss_exclude['date'])
combined_csv_ss_exclude = combined_csv_ss_exclude[[ 'amount', 'date']]
combined_csv_ss_exclude = combined_csv_ss_exclude.rename(columns=({'amount': 'Revenue','date': 'Date'}))
combined_csv_ss_exclude_11 = combined_csv_ss_exclude[combined_csv_ss_exclude['Date']>='2022-11-01'].reset_index(drop=True)
# case #4: push revenue 
email_pubid_int = emit['Revenue Pub ID'].unique().tolist()
combined_csv_ss_rest =  combined_csv[(combined_csv['first_split']!='SS') & (combined_csv['first_split']!='SMS') & (combined_csv['affiliate_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['site_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['first_split']!='MD') &  (combined_csv['first_split']!='TB')  ] 
#combined_csv_ss_rest =  combined_csv[(combined_csv['first_split']!='SS') & (combined_csv['first_split']!='SMS') & (combined_csv['subid_2'].str.contains('FLC|MBC',na = False)== False)& (combined_csv['affiliate_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['site_id'].astype(float).isin(email_pubid_int)== False) & (combined_csv['first_split']!='MD')   ] 

combined_csv_push = combined_csv_ss_rest.loc[combined_csv_ss_rest['affiliate_name'].str.lower().str.contains("push", na = False),]
combined_csv_push['date'] = pd.to_datetime(combined_csv_push['date'])
combined_csv_push = combined_csv_push[[ 'amount', 'date']]
combined_csv_push = combined_csv_push.rename(columns=({'amount': 'Revenue','date': 'Date'}))
combined_csv_push_11 =  combined_csv_push[combined_csv_push['Date']>='2022-11-01'].reset_index(drop=True)

# case #6: previos AR job
combined_csv_w1= combined_csv_ss_rest.loc[combined_csv_ss_rest['subid_2'].str.lower().str.contains("_w1", na = False),]
combined_csv_w1['Send Strategy'] = 'AR'
combined_csv_w1['Revenue Source'] = 'Short Code - SS Jobs'
combined_csv_w1['date'] = pd.to_datetime(combined_csv_w1['date'])
combined_csv_w1 = combined_csv_w1.rename(columns=({'amount': 'Revenue','date': 'Date','affiliate_id':'Affiliate_Id', 'subid_2':'Sub_Id'}))
combined_csv_w1['Hitpath Id'] = combined_csv_w1['campaign_id'].astype('str').str.split('.',expand = True)[0]
# split subid_2 with "_"
combined_csv_w1['Shortcode Name'] = np.nan
combined_csv_w1.loc[combined_csv_w1['Date']>= '2022-11-01','Shortcode Name'] = combined_csv_w1['Sub_Id'].str.split("_",expand = True)[2]
## when shortcode name is 460654
combined_csv_w1.loc[combined_csv_w1['Shortcode Name'] == '460654', 'Shortcode Name' ] = np.nan
## when shortcode name is PLV, it's long code 
combined_csv_w1 = combined_csv_w1[['Revenue', 'Date', 'Send Strategy', 'Revenue Source', 'Hitpath Id', 'Affiliate_Id', 'Sub_Id','Shortcode Name']]
combined_csv_w1_11 = combined_csv_w1[combined_csv_w1['Date']>='2022-11-01']
# case #5: revenue the rest of the revenue. 

combined_csv_ss_last =  combined_csv_ss_rest[ ((combined_csv['affiliate_name'].str.lower().str.contains("push", na = False) == False)) & (combined_csv_ss_rest['subid_2'].str.lower().str.contains("w1", na = False) == False)] 
combined_csv_ss_last_11 = combined_csv_ss_last[combined_csv_ss_last['date']>='2022-11-01'].reset_index(drop=True)

# identify Revenue Sourc
combined_csv_ss_creative_na['Revenue Source'] = 'Short Code - SS Jobs'
combined_csv_ss_creative_notna['Revenue Source'] = 'Short Code - SS Jobs'
combined_csv_ss_exclude['Revenue Source'] = 'Email'
combined_csv_push['Revenue Source'] = 'Push'
#historic_data['Revenue Source'] = 'Short Code - SS Jobs'
#historic_data['Send Strategy'] = np.nan

#combined_csv_ss_creative_na = combined_csv_ss_creative_na.drop_duplicates()
SS_New_data = pd.concat([combined_csv_ss_creative_notna,combined_csv_ss_creative_na,flows_clean2,combined_csv_ss_exclude, combined_csv_push,combined_csv_w1], axis=0)
SS_New_data['Ecpm'] = SS_New_data['Revenue'] * 1000/SS_New_data['Delivered']
SS_New_data['Roi'] = SS_New_data['Cost'] - SS_New_data['Revenue']
#SS_New_data['Sent'] = SS_New_data['Total']

#combining historic data(before Nov 1) with new data to get SS full data.
#SS_New_data = SS_New_data[historic_data.columns]


#SS_Full_data = pd.concat([SS_New_data,historic_data], axis = 0)
SS_Full_data = SS_New_data
SS_Full_data['Date'] = pd.to_datetime(SS_Full_data['Date'])
SS_Full_data = SS_Full_data.sort_values('Date', ascending=False)
#SS_Full_data.loc[(SS_Full_data['Job_Id'].isna()) & (SS_Full_data['Revenue Source']==  'Short Code - SS Jobs')  ,'Revenue Source'] = 'Short Code - Opt In'
SS_Full_data.loc[ (SS_Full_data['Job_Id'].isna()) & (SS_Full_data['Sub_Id'].str.contains('AR', na = False)) ,'Revenue Source'] =  'Short Code - SS Flow'
SS_Full_data.loc[ (SS_Full_data['Job_Id'].isna())&  (SS_Full_data['Sub_Id'].str.contains('AR', na = False)) ,'Send Strategy'] = 'AR'
SS_Full_data.loc[(SS_Full_data['Job_Id'].isna()) & (SS_Full_data['Revenue Source']==  'Short Code - Opt In') ,'Send Strategy'] = 'Opt In'
SS_Full_data.loc[(SS_Full_data['Job_Id']==0) & (SS_Full_data['Revenue Source']==  'Short Code - SS Jobs') & (SS_Full_data['Send Strategy'].isna()) ,'Send Strategy'] = 'AR'
SS_Full_data.to_csv(localfolder + 'SS_Fulldata.csv', index =False)  

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\2422394425.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_ss_exclude['date'] = pd.to_datetime(combined_csv_ss_exclude['date'])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\2422394425.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_csv_push['date'] = pd.to_datetime(combined_csv_push['date'])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\2422394425.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy

## long code MP 

In [15]:
# ######## MP BEGIN ########
# #Read MP file and clean. All stats present in MP_Campaigns file.

# MP_campaigns = pd.read_csv(localfolder + 'SMS_SC_MP_Campaigns.csv')
# MP_campaigns = MP_campaigns[~MP_campaigns['ACTDATE'].isna()]
# MP_campaigns = MP_campaigns[~MP_campaigns['ACTDATE'].isin(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])]
# MP_campaigns['ACTDATE']= pd.to_datetime(MP_campaigns['ACTDATE'])
# MP_campaigns = MP_campaigns[MP_campaigns['ACTDATE'] > two_months_ago]
# #a = MP_campaigns.isna().sum()
# drop_columns = ['Daily Opt Out', 'Unnamed: 28', 'Unnamed: 53', 'Reference', 'c1NEW' , 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 64', 'Unnamed: 72', 'Unnamed: 73']
# MP_campaigns.drop(columns=drop_columns, inplace=True, axis=1)
# MP_campaigns= MP_campaigns[~MP_campaigns['Done.'].isna()]
# MP_campaigns.columns = MP_campaigns.columns.str.strip()

# # remove % and $ symbold
# MP_campaigns['REV'] = MP_campaigns['REV'].str.strip()
# MP_campaigns['REV'] = MP_campaigns['REV'].replace('$ -', np.nan)
# MP_campaigns['REV'] = MP_campaigns['REV'].str.replace('$','')
# MP_campaigns['REV'] = MP_campaigns['REV'].str.replace(',','')
# MP_campaigns['REV'] = MP_campaigns['REV'].astype('float')

# MP_campaigns['COST'] = MP_campaigns['COST'].str.strip()
# MP_campaigns['COST'] = MP_campaigns['COST'].replace('$ -', np.nan)
# MP_campaigns['COST'] = MP_campaigns['COST'].str.replace('$','')
# MP_campaigns['COST'] = MP_campaigns['COST'].astype('float')

# MP_campaigns['eCPM'] = MP_campaigns['eCPM'].str.strip()
# MP_campaigns['eCPM'] = MP_campaigns['eCPM'].replace('$ -', np.nan)
# MP_campaigns['eCPM'] = MP_campaigns['eCPM'].str.replace('$','')
# MP_campaigns['eCPM'] = MP_campaigns['eCPM'].astype('float')

# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].str.strip()
# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].str.replace('$','')
# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].replace('-',np.nan)
# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].replace(' -',np.nan)
# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].astype(str).str.replace('\((.*)\)', '-\\1')
# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].str.replace(',','')
# MP_campaigns['gPROFIT'] = MP_campaigns['gPROFIT'].astype('float')

# MP_campaigns['CLICK %'] = MP_campaigns['CLICK %'].str.strip()
# MP_campaigns['CLICK %'] = MP_campaigns['CLICK %'].str.replace('%','')
# MP_campaigns['CLICK %'] = MP_campaigns['CLICK %'].astype('float')

# MP_campaigns['gMARGIN'] = MP_campaigns['gMARGIN'].str.strip()
# MP_campaigns['gMARGIN'] = MP_campaigns['gMARGIN'].str.replace('%','')
# MP_campaigns['gMARGIN'] = MP_campaigns['gMARGIN'].astype('float')

# #MP_campaigns.to_csv(localfolder + 'MP_data.csv', index =False)  

## Combine all data from all sources 

In [16]:
#4 merge SS and MP: not possible as there are columns in MP not in SS.
#LC_data = pd.read_csv(localfolder + 'SMS_LC_Campaigns_clean.csv')
SS_data = pd.read_csv(localfolder + 'SS_Fulldata.csv')

# lc_df_full = lc_df_full.reset_index(drop=True)
# LC_data = LC_data.reset_index(drop=True)
# LC_data = pd.concat([lc_df_full,LC_data], axis=0, ignore_index=True)
# LC_data.loc[LC_data['Affiliate_Id'].isna(),'Affiliate_Id'] = LC_data['pubID']
# LC_data.loc[LC_data['Hitpath_Offer_ID'].isna(),'Hitpath_Offer_ID' ] = LC_data['sid']
# LC_data['Affiliate_Id'] = LC_data['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
# LC_data['Hitpath_Offer_ID'] = LC_data['Hitpath_Offer_ID'].astype(str).str.split('.',expand = True)[0]
SS_data['Code_Type'] = 'Short Code'
# LC_data['Code_Type'] = 'Long Code'
SS_data.loc[SS_data['Revenue Source'] == 'Email', 'Code_Type'] = 'Email'
SS_data.loc[SS_data['Revenue Source'] == 'Push', 'Code_Type'] = 'Push'
########Newly added
SS_data.loc[SS_data['Shortcode'].astype(str).str.replace('.0','').str.len() ==11, 'Code_Type'] = 'Long Code'
SS_data.loc[
    (SS_data['Shortcode'].astype(str).str.replace('.0','').str.len() == 11) &
    (SS_data['Shortcode'].astype(str).str.startswith('183')),
    'Code_Type'
] = 'Toll Free'

SS_data.loc[SS_data['Shortcode'].astype(str).str.replace('.0','').str.len() ==11, 'Revenue Source'] = 'Long Code'
SS_data.loc[
    (SS_data['Shortcode'].astype(str).str.replace('.0','').str.len() == 11) &
    (SS_data['Shortcode'].astype(str).str.startswith('183')),
    'Revenue Source'
] = 'Toll Free'

SS_data['Hitpath_Offer_ID'] = SS_data['Offer'].astype('str').str.extract(r'\b(\d{4,5}).*')
SS_data.loc[SS_data['Hitpath_Offer_ID'].isna(),'Hitpath_Offer_ID'] = SS_data['Hitpath Id'].astype(str).str.split('.',expand = True)[0]
SS_data['Affiliate_Id'] = SS_data['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
#LC_data= LC_data.rename(columns=({'eCPM' : 'Ecpm', 'Clicked': 'Clicks'}))


#merge short code and longcode
#merged_data = pd.concat([SS_data,LC_data], axis=0, ignore_index=True)
merged_data =SS_data

# merge offer Wall 

merged_data['Hitpath_Offer_ID'] = merged_data['Hitpath_Offer_ID'].astype(str).str.split('.',expand = True)[0]
merged_data.loc[merged_data['Revenue Source']=='Short Code - SS Flow','Hitpath_Offer_ID'] =merged_data['Hitpath Id'].astype('str').str.split('.',expand = True)[0]
merged_data['Offer Wall Id'] = merged_data['Offer Wall Id'].astype('str').str.split('.',expand = True)[0]
merged_data.loc[ (merged_data['Offer Wall Id'].isin(offer_wall_id)),'Hitpath_Offer_ID'] = merged_data['Offer Wall Id']
merged_data['Offer Type'] =  'Single Offer'
merged_data.loc[ (merged_data['Offer Wall Id'].isin(offer_wall_id)),'Offer Type'] = 'Offer Wall'
merged_data['DP.SV'] = merged_data['Dp.Ds Or Dp.Sv']
merged_data.loc[merged_data['DP.SV'].isnull(), 'DP.SV'] =  merged_data['Segments'].str.split('_',expand = True)[1]
dict_publisher = {'DP.DS or DP.sV':['WWM.YFA','ZM.PL'], 'PUBID': ['461680','461681']}
dict_publisher = pd.DataFrame(dict_publisher)
new_publisher = pd.concat([publisher,dict_publisher])
new_publisher = new_publisher.reset_index(drop=True)
# publisher drop na rows
new_publisher = new_publisher[~new_publisher['DP.DS or DP.sV'].isna()]
new_publisher = new_publisher[['DP.DS or DP.sV','PUBID']]
new_publisher['PUBID'] = new_publisher['PUBID'].astype(str).str.split('.',expand = True)[0]
merged_data = merged_data.merge(new_publisher[['DP.DS or DP.sV','PUBID']], left_on ='DP.SV', right_on = 'DP.DS or DP.sV', how = 'left' )
merged_data.loc[(merged_data['Affiliate_Id'].isna()) |  (merged_data['Affiliate_Id']=='nan'), 'Affiliate_Id'] = merged_data['PUBID']
print(merged_data['Revenue'].sum())
publisher_copy = publisher
publisher_copy['DP.DS or DP.sV'] = publisher_copy['DP.DS or DP.sV'].str.replace('WWM.YFA','WWM.YFA.2')
publisher_copy['DP.DS or DP.sV'] = publisher_copy['DP.DS or DP.sV'].str.replace('ZM.PL','ZM.PL.2')
publisher_copy = publisher_copy[['DP.DS or DP.sV','PUBID']].drop_duplicates()
merged_data = merged_data.merge(publisher_copy, left_on ='Affiliate_Id', right_on = 'PUBID', how = 'left')
#merged_data = merged_data.merge(new_publisher[['DP.DS or DP.sV','PUBID']], left_on ='Affiliate_Id', right_on = 'PUBID', how = 'left' )
merged_data.loc[(merged_data['DP.SV'].isna()) |  (merged_data['DP.SV']=='nan'),'DP.SV'] = merged_data['DP.DS or DP.sV_y']
print(merged_data['Revenue'].sum())
merged_data['Affiliate_Id'] = merged_data['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
merged_data['DP&Pub'] = merged_data['DP.SV']+'_'+ merged_data['Affiliate_Id']

##add conversions
merged_data = merged_data[['Date','Scheduling Time', 'Offer','Hitpath_Offer_ID','DP.SV','Affiliate_Id', 'DP&Pub','Job_Id', 'Job_Name','Creative_Id',
                           'Creativename','Creative','Send Strategy', 'Shortcode', 'Start_Tstamp','Segments', 'Revenue','Conversions','Jump Page Clicks', 
                           'Delivered', 'Optout', 'Clicks','Cost', 'Ecpm', 'Dataset', 'Code_Type','Revenue Source','Ar Day','Sub_Id','Campaign_Id',
                           'Roi','Shortcode Name','Jump Page Version','Offer Wall Id','Offer Type','Flow_Abbr']]
merged_data.loc[merged_data['Clicks'] == '','Clicks'] = np.nan
merged_data['Clicks'] = merged_data['Clicks'].astype(float)
merged_data.loc[merged_data['Shortcode'] == 51797, 'Shortcode Name'] = 'FLC'
merged_data.loc[merged_data['Shortcode'] == 70610, 'Shortcode Name'] = 'CSS'
merged_data.loc[merged_data['Shortcode'] == 44345, 'Shortcode Name'] = 'HZB'
merged_data.loc[merged_data['Shortcode'] == 80837, 'Shortcode Name'] = 'MBC'
merged_data.loc[merged_data['Shortcode'] == 31232, 'Shortcode Name'] = 'DSS'
merged_data.loc[merged_data['Shortcode'] == 61659, 'Shortcode Name'] = 'SVT'
merged_data.loc[merged_data['Shortcode'] == 79743, 'Shortcode Name'] = 'UAA'
merged_data.loc[merged_data['Shortcode'] == 18333164159, 'Shortcode Name'] = 'PRC'
merged_data.loc[merged_data['Shortcode'] == 18333641722, 'Shortcode Name'] = 'UAATF'
merged_data.loc[merged_data['Shortcode'] == 13022952715, 'Shortcode Name'] = 'A4FLC'
merged_data.loc[merged_data['Shortcode'] == 15612023538, 'Shortcode Name'] = 'A4FLC'
merged_data.loc[merged_data['Shortcode'] == 18332686782, 'Shortcode Name'] = 'MFATF'
#added
merged_data.loc[merged_data['Shortcode'] == 18332685688, 'Shortcode Name'] = 'N3GTF'
merged_data.loc[merged_data['Shortcode'] == 18338987652, 'Shortcode Name'] = 'FBHTF'
merged_data.loc[merged_data['Shortcode'] == 18333275117, 'Shortcode Name'] = 'FRHTF'
merged_data.loc[merged_data['Shortcode Name'] == 'FLC', 'Shortcode'] = '51797'
merged_data.loc[merged_data['Shortcode Name'] == 'CSS', 'Shortcode'] = '70610'
merged_data.loc[merged_data['Shortcode Name'] == 'HZB', 'Shortcode'] = '44345'
merged_data.loc[merged_data['Shortcode Name'] == 'MBC', 'Shortcode'] = '80837'
merged_data.loc[merged_data['Shortcode Name'] == 'DSS', 'Shortcode'] = '31232'
merged_data.loc[merged_data['Shortcode Name'] == 'SVT', 'Shortcode'] = '61659'
merged_data.loc[merged_data['Shortcode Name'] == 'UAA', 'Shortcode'] = '79743'
merged_data.loc[merged_data['Shortcode Name'] == 'PRC', 'Shortcode'] = '18333164159'
merged_data.loc[merged_data['Shortcode Name'] == 'UAATF', 'Shortcode'] = '18333641722'
merged_data.loc[merged_data['Shortcode Name'] == 'A4FLC', 'Shortcode'] = '13022952715'
merged_data.loc[merged_data['Shortcode Name'] == 'MFATF', 'Shortcode'] = '18332686782'
#added
merged_data.loc[merged_data['Shortcode Name'] == 'N3GTF', 'Shortcode'] = '18332685688'
merged_data.loc[merged_data['Shortcode Name'] == 'FBHTF', 'Shortcode'] = '18338987652'
merged_data.loc[merged_data['Shortcode Name'] == 'FRHTF', 'Shortcode'] = '18333275117'
merged_data.loc[merged_data['Delivered'] == '','Delivered'] = np.nan
merged_data['Delivered'] =  merged_data['Delivered'].astype(float)
merged_data.loc[(merged_data['Shortcode Name'] == 'HZB'), 'Cost' ] = 0.007 * merged_data['Delivered']
merged_data.loc[(merged_data['Shortcode Name'] == 'MBC') | (merged_data['Shortcode Name'] == 'FLC'),'Cost']  =  0.00563 *  merged_data['Delivered'] +  merged_data['Cost']
merged_data.loc[(merged_data['Code_Type'] == 'Short Code')&((merged_data['Shortcode Name'] == 'UAA') | (merged_data['Shortcode Name'] == 'SVT') | (merged_data['Shortcode Name'] == 'DSS')), 'Cost']  =  0.00504 *  merged_data['Delivered'] 
# we need to add sending cost for MFATF and A4FLC
Offer_name[ 'campaign_id'] = Offer_name[ 'campaign_id'].astype('str').str.split('.',expand = True)[0]
merged_data = merged_data.merge(Offer_name,left_on = 'Hitpath_Offer_ID',right_on = 'campaign_id',how = 'left')
merged_data['Job_Id']  = merged_data['Job_Id'].astype(str).str.split('.',expand = True)[0].str.strip()
merged_data['Date'] = pd.to_datetime(merged_data['Date'],format ='mixed').dt.strftime("%Y-%m-%d")
merged_data['shortcode_DP.SV'] = merged_data['Shortcode Name'] + "_" + merged_data['DP.SV']
merged_data.loc[merged_data['shortcode_DP.SV']=='_', 'shortcode_DP.SV'] = np.nan
# calculate opportunity cost
merged_data['Opportunity Cost Send Strategy'] =  True
merged_data.loc[merged_data['Send Strategy'].isin(['Null','Opt In',np.nan]), 'Opportunity Cost Send Strategy'] = False
merged_data = merged_data.sort_values('Date')
temp1= merged_data.groupby(['shortcode_DP.SV','Opportunity Cost Send Strategy','Date']).agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
temp1[['rolling Revenue','rolling Delivered']] = temp1.groupby('shortcode_DP.SV').shift(1).rolling(30, min_periods=5)[['Revenue','Delivered']].sum().reset_index(drop=True)
temp1['Dataset_Agg_30D_eCPM'] = temp1['rolling Revenue'] * 1000/ temp1['rolling Delivered']
dataset_agg_eCPM =  temp1[['shortcode_DP.SV','Date','Opportunity Cost Send Strategy','Dataset_Agg_30D_eCPM']]
merged_data = merged_data.merge(dataset_agg_eCPM, how = 'left')
merged_data['Opportunity Cost'] = merged_data['Revenue'] - merged_data['Dataset_Agg_30D_eCPM'] * merged_data['Delivered'] /1000 

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\689150103.py:3: DtypeWarning: Columns (0,4,5,6,7,15,17,18,19,20,21,22,24,27,29,34,35,36,37,39,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  SS_data = pd.read_csv(localfolder + 'SS_Fulldata.csv')


374429.14589800005
374429.14589800005


In [17]:
# combined_df["Hitpath_Offer_ID"].unique()

In [18]:
print(merged_data['Revenue'].sum())

374429.14589800005


In [19]:
# Add offer & vertical gap
oms = infrastructure.get_smartsheet('offers_sms')
oms.rename(columns ={'Hitpath Offer ID':'Hitpath_Offer_ID'}, inplace = True)
oms.rename(columns ={"Vertical": "Offer Vertical"}, inplace = True)
offer_vertical=oms[['Hitpath_Offer_ID','Offer Vertical']]
offer_vertical = offer_vertical.dropna(subset=['Hitpath_Offer_ID'])
offer_vertical['Hitpath_Offer_ID'] = offer_vertical['Hitpath_Offer_ID'].astype(str).replace(r'\.0$','', regex=True)
merged_data= merged_data.merge(offer_vertical, how = 'left', on = "Hitpath_Offer_ID")
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data.sort_values(by = ['shortcode_DP.SV','Hitpath_Offer_ID','Send Strategy','Date'], ascending = True, inplace = True)
merged_data['Offer Gap'] = merged_data.groupby(['shortcode_DP.SV','Send Strategy','Hitpath_Offer_ID'])['Date'].diff()
merged_data['Vertical Gap'] = merged_data.groupby(['shortcode_DP.SV','Send Strategy',"Offer Vertical"])['Date'].diff()
merged_data['Vertical Gap'] = merged_data['Vertical Gap'].dt.days
merged_data['Offer Gap'] = merged_data['Offer Gap'].dt.days
merged_data.loc[merged_data['Send Strategy'] != 'P', 'Offer Gap'] = np.nan
merged_data.loc[merged_data['Send Strategy'] != 'P', 'Vertical Gap'] = np.nan

In [20]:
#added ecpm ratio 
merged_data.replace([np.inf, -np.inf], np.nan, inplace=True)
merged_data['Dataset_Agg_30D_eCPM'] = merged_data['Dataset_Agg_30D_eCPM'].replace(0, np.nan)
merged_data['eCPM ratio'] = merged_data['Ecpm']/merged_data['Dataset_Agg_30D_eCPM']

In [21]:
#Combine history data, sort date and export both SS_LC_merged_data and data from two months ago
twomonthsago = pd.read_csv(localfolder + 'SS_LC_merged_data_twomonthsago.csv', index_col= False)
twomonthsago['Hitpath_Offer_ID'] = twomonthsago['Hitpath_Offer_ID'].astype(str).str.replace('.0', '')
combined_df = pd.concat([twomonthsago, merged_data], ignore_index=True)

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\3635888808.py:2: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  twomonthsago = pd.read_csv(localfolder + 'SS_LC_merged_data_twomonthsago.csv', index_col= False)


In [22]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df = combined_df.sort_values(by = 'Date',ascending = False)
combined_df.to_csv(localfolder + 'SS_LC_merged_data.csv', index =False) 

In [23]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
twomonthsago_new = combined_df[(combined_df['Date'].isna()) | (combined_df['Date'] <= two_months_ago)]
twomonthsago_new.to_csv(localfolder + 'SS_LC_merged_data_twomonthsago.csv', index =False)

# P&L DATA 

In [24]:
merged_data = pd.read_csv(localfolder + 'SS_LC_merged_data.csv')
engagement_daily = merged_data.groupby(['Date','Affiliate_Id'])[['Delivered','Clicks','Jump Page Clicks','Optout','Cost']].sum()
engagement_daily = engagement_daily[~np.all(engagement_daily==0,axis = 1)]
engagement_daily = engagement_daily.reset_index()
daily_revenue_gpby = daily_revenue_gpby.rename(columns = {'amount':'Revenue','affiliate_id': 'Affiliate_Id', 'date': 'Date'})
#Pl_data = daily_revenue_gpby.merge(engagement_daily, left_on =['affiliate_id','date'] ,right_on = ['Affiliate_Id','Date'],copy = False, how = 'outer')
daily_revenue_gpby['Date'] = pd.to_datetime(daily_revenue_gpby['Date'])
engagement_daily['Date'] =  pd.to_datetime(engagement_daily['Date'])
Pl_data = pd.concat([daily_revenue_gpby,engagement_daily], axis=0, ignore_index=True)
Pl_data['PUBID'] =Pl_data['Affiliate_Id'].astype('str').str.split(".",expand = True)[0]
Pl_data = Pl_data.fillna(0)
Pl_data = Pl_data[[ 'Date', 'PUBID','Revenue', 'Delivered', 'Clicks',\
      'Jump Page Clicks', 'Optout','Cost']]
Pl_data = Pl_data.groupby(['PUBID','Date']).sum().reset_index()
#Pl_data.to_csv(localfolder+'p&l_data.csv',index =False)

api_key = pd.read_csv(localfolder+'SMS_SC_SS_Apikey.csv')
#publisher1 = publisher
#publisher1['DP.DS or DP.sV'] = publisher1['DP.DS or DP.sV'].str.replace('WWM.YFA.2','WWM.YFA')
#publisher1['DP.DS or DP.sV'] = publisher1['DP.DS or DP.sV'].str.replace('ZM.PL.2','ZM.PL')
api_key_sms = api_key.merge(new_publisher, left_on = 'DP.SV',right_on ='DP.DS or DP.sV',how = 'inner' )
api_key_sms = api_key_sms[['DP.SV','PUBID','Date', 'AcceptedTotal', 'AcceptedNew',\
       'AcceptedDuplicate', 'RejectedTotal', 'RejectedMobile',\
       'RejectedBlacklist', 'RejectedData', 'CostData', 'CostChecks',\
       'CostTotal']]
api_key_sms['Date'] = pd.to_datetime(api_key_sms['Date'])
Pl_data['Date'] =  pd.to_datetime(Pl_data['Date'] )
Pl_data['PUBID']  = Pl_data['PUBID'].astype('str').str.split(".",expand = True)[0]
api_key_sms['PUBID']  = api_key_sms['PUBID'].astype('str').str.split(".",expand = True)[0]
rev_accept_df = pd.concat([Pl_data, api_key_sms], axis=0, ignore_index=True)
rev_accept_df = rev_accept_df.fillna(0)
rev_accept_df['PUBID']  = rev_accept_df['PUBID'].astype('str').str.split(".",expand = True)[0]
rev_accept_df = rev_accept_df.drop(columns = 'DP.SV')
rev_accept_df = rev_accept_df.groupby(['PUBID','Date']).sum().reset_index()


sms_post = pd.read_csv(localfolder + 'SMSposts.csv')
sms_post['Date'] = pd.to_datetime(sms_post['Dates'])
sms_post['PUBID'] = sms_post['Pub Id'].astype('str').str.split(".",expand = True)[0]
sms_post = sms_post.drop(columns = ['Dates','Pub Id','Pub Name'])
post_data = pd.concat([rev_accept_df,sms_post], axis=0, ignore_index=True)
post_data = post_data.fillna(0)
post_data = post_data.groupby(['PUBID','Date']).sum().reset_index()
publisher_raw['PUBID'] = publisher_raw['PUBID'].astype('str').str.split(".",expand = True)[0]
post_data = post_data.merge(publisher_raw[['PUBID','PUBLISHER' ,'DP.DS or DP.sV','DMA', 'INTERNAL STATUS',
       'DATA TEAM STATUS', 'Sub Vertical', 'SCOPE', 'COMPANY (DP)','DP ID']], how = 'left', on = 'PUBID')
post_data.to_csv(localfolder+'p&l_data.csv',index =False)



D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\4062370451.py:1: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv(localfolder + 'SS_LC_merged_data.csv')


In [25]:
# offerwall reporting 
import infrastructure 
offer_sheet = infrastructure.get_smartsheet('offers_sms')
offer_sheet = offer_sheet[offer_sheet['Hitpath Offer ID'].isna() == False]
offer_sheet.rename(columns ={'Hitpath Offer ID':'Hitpath ID'}, inplace = True)
offer_sheet['Hitpath ID'] = offer_sheet['Hitpath ID'].astype(str).str.split(".",expand = True)[0]
offerwall = offerwall.rename(columns = {'offer_id':'Offer Wall ID'})
print(offerwall[ 'amount'].sum())
offerwall_update = offerwall.merge(offer_sheet[['Hitpath ID','Scheduling Name']], copy = False, how = 'left', left_on = 'Offer Wall ID', right_on = 'Hitpath ID')
print(offerwall_update[ 'amount'].sum())
offerwall_update = offerwall_update.rename(columns = {'Scheduling Name':"Offer Wall Scheduling Name" })
offerwall_update = offerwall_update.merge(offer_sheet[['Hitpath ID','Scheduling Name']], copy = False, how = 'left', left_on = 'campaign_id', right_on = 'Hitpath ID')
offerwall_update.loc[offerwall_update['Scheduling Name'].isna(),'Scheduling Name' ] = offerwall_update['campaign_name']
offerwall_update = offerwall_update.rename(columns = {'Scheduling Name':"Child Offer Campaign Name" })

offerwall_update[['Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]] = offerwall_update[['Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]].fillna("NULL")
offerwall_rev = offerwall_update.groupby(['Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]).agg({
            'amount': ['sum', 'count'],  # Summing and counting for the 'amount' column
            'Jump Page Clicks': 'sum'     # Summing for the 'Jump Page Clicks' column
        }).reset_index()
 
offerwall_rev.columns = [
'Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id',"Child Offer Campaign Name",'sub_id','Offer Wall ID',"Offer Wall Scheduling Name",'amount','conversions','Jump Page Clicks'
]
print(offerwall_rev[ 'amount'].sum())
offerwall_rev['split_column'] = offerwall_rev['sub_id'].str.split('_')
offerwall_rev['Send Strategy'] = 'P'
offerwall_rev.loc[offerwall_rev['split_column'].str[5].str.contains("AR",na = False),'Send Strategy'] = 'AR'
offerwall_rev.loc[offerwall_rev['split_column'].str[5].str.contains("CT",na = False),'Send Strategy'] = 'CT'
offerwall_rev = offerwall_rev.rename(columns = {'amount':'Child Offer Revenue','Jump Page Clicks':'Child Offer Jump Page Clicks', 'campaign_id':'Child Offer ID'})
merged_data = pd.read_csv(localfolder + 'SS_LC_merged_data.csv')
offerwall_engage_merge = merged_data[(merged_data['Offer Type'] == 'Offer Wall')]
offerwall_engage_merge = offerwall_engage_merge.add_suffix('- Offer Wall')
# The delivered volume is biased, if we don't have conversions & jump page clicks for a child offer, then it won't have the delivered and engagement stat.  
offerwall_engage_subid = offerwall_engage_merge.groupby(['Sub_Id- Offer Wall'])[['Delivered- Offer Wall','Clicks- Offer Wall','Optout- Offer Wall','Cost- Offer Wall','Revenue- Offer Wall','Jump Page Clicks- Offer Wall']].sum().reset_index()
offerwall_engage =  offerwall_engage_merge.groupby(['Date- Offer Wall','Affiliate_Id- Offer Wall','Hitpath_Offer_ID- Offer Wall','Send Strategy- Offer Wall'])[['Delivered- Offer Wall','Clicks- Offer Wall','Optout- Offer Wall','Cost- Offer Wall','Revenue- Offer Wall','Jump Page Clicks- Offer Wall']].sum().reset_index()
offerwall_rev['affiliate_id'] = offerwall_rev['affiliate_id'].astype(str).str.split('.',expand = True)[0]
offerwall_rev['Offer Wall ID'] = offerwall_rev['Offer Wall ID'].astype(str).str.split('.',expand = True)[0]
offerwall_engage['Date- Offer Wall'] = pd.to_datetime(offerwall_engage['Date- Offer Wall'])
offerwall_rev['Dash_Date_from_subid'] = pd.to_datetime(offerwall_rev['Dash_Date_from_subid'])
"""
# offer wall list 
temp = ofwall_df[['Hitpath Offer ID','Offer List' ]]
temp['Offer List'] = temp['Offer List'].str.replace("[","").str.replace("]","").str.split(",")
offerwall_match = temp.explode('Offer List')
offerwall_match['Offer List'] = offerwall_match['Offer List'].str.strip()
offerwall_engage_subid = offerwall_engage.merge(offerwall_match, left_on = 'Hitpath_Offer_ID- Offer Wall', right_on = 'Hitpath Offer ID', how = 'left')
""" 
# merge rev, engagement data together
offerwall_performance = offerwall_rev.merge(offerwall_engage_subid, left_on = ['sub_id'], right_on =['Sub_Id- Offer Wall'], how = 'left' )
offerwall_performance_1 = offerwall_performance.loc[offerwall_performance['Sub_Id- Offer Wall'].isna() == False]
offerwall_performance_2 = offerwall_performance.loc[offerwall_performance['Sub_Id- Offer Wall'].isna()== True].drop(columns = ['Sub_Id- Offer Wall','Delivered- Offer Wall','Clicks- Offer Wall','Optout- Offer Wall','Cost- Offer Wall','Revenue- Offer Wall','Jump Page Clicks- Offer Wall'])
offerwall_performance_2 = offerwall_performance_2.groupby(['Dash_Date_from_subid','affiliate_id','Offer Wall ID','Send Strategy','Child Offer ID'])[[ 'Child Offer Revenue', 'Child Offer Jump Page Clicks']].sum().reset_index()
offerwall_engage['Hitpath_Offer_ID- Offer Wall'] = offerwall_engage['Hitpath_Offer_ID- Offer Wall'].astype(str).str.split(".",expand = True)[0]
offerwall_engage['Affiliate_Id- Offer Wall'] = offerwall_engage['Affiliate_Id- Offer Wall'].astype(str).str.split(".",expand = True)[0]
offerwall_performance_2 = offerwall_performance_2.merge(offerwall_engage, left_on = ['Dash_Date_from_subid','affiliate_id','Offer Wall ID','Send Strategy'], right_on = ['Date- Offer Wall','Affiliate_Id- Offer Wall','Hitpath_Offer_ID- Offer Wall','Send Strategy- Offer Wall'], how ='left')
offerwall_report = pd.concat([offerwall_performance_1,offerwall_performance_2])
offerwall_report = offerwall_performance_2.merge(ofwall_df, left_on = 'Offer Wall ID', right_on = 'Offer Wall ID', how = 'left')
print(offerwall_report['Child Offer Revenue'].sum())
offerwall_report.to_csv(localfolder + "offerwall_data.csv")


7141.0
7141.0
7141.000000000001


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\4158485992.py:31: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv(localfolder + 'SS_LC_merged_data.csv')


7010.65


# Offerwall_rev

In [26]:

import pandas as pd

# Second Approach to make offer wall report 
################

offer_sheet = infrastructure.get_smartsheet('offers_sms')
offer_sheet = offer_sheet[offer_sheet['Hitpath Offer ID'].isna() == False]
offer_sheet['Hitpath Offer ID'] = offer_sheet['Hitpath Offer ID'].astype(str).str.split(".",expand = True)[0]
offerwall = offerwall.rename(columns = {'offer_id':'Offer Wall ID'})
offerwall.loc[offerwall['campaign_id']=='nan','campaign_id'] = offerwall['Offer Wall ID']
offerwall.loc[(offerwall['Offer Type']!='Offer Wall') & (offerwall['Offer Wall ID']!= offerwall['campaign_id']) ,'Offer Type'] = 'JP Extra Rev'


offerwall_update = offerwall.merge(offer_sheet[['Hitpath Offer ID','Scheduling Name']], copy = False, how = 'left', left_on = 'Offer Wall ID', right_on = 'Hitpath Offer ID')
print(offerwall_update[ 'amount'].sum())
offerwall_update = offerwall_update.rename(columns = {'Scheduling Name':"Offer Wall Scheduling Name" })
#offerwall_update = offerwall_update.merge(offer_sheet[['Hitpath Offer ID','Scheduling Name']], copy = False, how = 'left', left_on = 'campaign_id', right_on = 'Hitpath Offer ID')
#offerwall_update.loc[offerwall_update['Scheduling Name'].isna(),'Scheduling Name' ] = offerwall_update['campaign_name']
#offerwall_update = offerwall_update.rename(columns = {'Scheduling Name':"Child Offer Campaign Name" })

offerwall_update[['Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id','sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]] = offerwall_update[['Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id','sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]].fillna("NULL")
offerwall_rev = offerwall_update.groupby(['Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id','sub_id','Offer Wall ID',"Offer Wall Scheduling Name" ]).agg({
            'amount': ['sum', 'count'],  # Summing and counting for the 'amount' column
            'Jump Page Clicks': 'sum'     # Summing for the 'Jump Page Clicks' column
        }).reset_index()
 
offerwall_rev.columns = [
'Dash_Date_from_subid','affiliate_id','Offer Type','campaign_id','sub_id','Offer Wall ID',"Offer Wall Scheduling Name",'amount','conversions','Jump Page Clicks'
]
print(offerwall_rev[ 'amount'].sum())
offerwall_rev['split_column'] = offerwall_rev['sub_id'].str.split('_')
offerwall_rev['Send Strategy'] = 'P'
offerwall_rev.loc[offerwall_rev['split_column'].str[5].str.contains("AR",na = False),'Send Strategy'] = 'AR'
offerwall_rev.loc[offerwall_rev['split_column'].str[5].str.contains("CT",na = False),'Send Strategy'] = 'CT'
offerwall_rev = offerwall_rev.rename(columns = {'amount':'Child Offer Revenue','Jump Page Clicks':'Child Offer Jump Page Clicks', 'campaign_id':'Child Offer ID'})

#################
merged_data_ow = pd.read_csv(localfolder + 'SS_LC_merged_data.csv')
merged_data_ow['data type'] = 'Total'
# offer type = offer wall 
merged_data_ow_only = merged_data_ow.loc[merged_data_ow['Offer Type'] == 'Offer Wall', ]
print("Offer Wall Revenue",str(merged_data_ow_only['Revenue'].sum()))
print("Offer Wall Delivered",str(merged_data_ow_only['Delivered'].sum()))
print("Offer Wall Revenue", str(offerwall_rev[offerwall_rev['Offer Type'] == 'Offer Wall']['Child Offer Revenue'].sum()))
merged_data_ow_only.drop(columns = ['Revenue','Jump Page Clicks'], inplace = True)
#all_ow_drops = merged_data_ow_only.groupby(['Date','Affiliate_Id','Hitpath_Offer_ID','Send Strategy']).count().reset_index()[['Date','Affiliate_Id','Hitpath_Offer_ID','Send Strategy']]
# offer wall list 
temp = ofwall_df[['Offer Wall ID','Offer List' ]]
temp['Offer List'] = temp['Offer List'].str.replace("[","").str.replace("]","").str.split(",")
offerwall_match = temp.explode('Offer List')
offerwall_match['Offer List'] = offerwall_match['Offer List'].str.strip()
offerwall_match.columns = ['Hitpath Offer ID','Child Offer ID']
offerwall_match['Child Offer Count'] = offerwall_match.groupby('Hitpath Offer ID')['Child Offer ID'].transform('count')
# divide mertics by child offer count
merged_data_ow_only = merged_data_ow_only.merge(offerwall_match,left_on = 'Hitpath_Offer_ID', right_on = 'Hitpath Offer ID', how = 'left')
merged_data_ow_only['Adjusted Delivered'] = merged_data_ow_only['Delivered'] / merged_data_ow_only['Child Offer Count']
merged_data_ow_only['Adjusted Optout'] = merged_data_ow_only['Optout'] / merged_data_ow_only['Child Offer Count']
merged_data_ow_only['Adjusted Clicks'] = merged_data_ow_only['Clicks'] / merged_data_ow_only['Child Offer Count']
merged_data_ow_only['Adjusted Cost'] = merged_data_ow_only['Cost'] / merged_data_ow_only['Child Offer Count']
print('Delivery',merged_data_ow_only['Adjusted Delivered'].sum())
# clean offerwall_rev
offerwall_rev.drop(columns =[ 'split_column','Offer Wall Scheduling Name','conversions'], inplace = True)
offerwall_rev.columns = ['Date','Affiliate_Id','Offer Type','Child Offer ID','Sub_Id','Hitpath_Offer_ID','Revenue','Jump Page Clicks','Send Strategy']
# change data type
merged_data_ow_only['Date'] = pd.to_datetime(merged_data_ow_only['Date'])
offerwall_rev['Date'] = pd.to_datetime(offerwall_rev['Date'])
merged_data_ow_only['Affiliate_Id'] = merged_data_ow_only['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
offerwall_rev['Affiliate_Id'] = offerwall_rev['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
offerwall_rev_only = offerwall_rev[offerwall_rev['Offer Type'] == 'Offer Wall'].drop(columns = ['Offer Type'])
# merge subid then hitpath offer id
#offerwall_engage_subid = merged_data_ow_only.groupby(['Sub_Id','Child Offer ID'])[['Delivered','Clicks','Optout','Cost','Adjusted Delivered','Adjusted Optout','Adjusted Clicks','Adjusted Cost']].sum().reset_index()
offerwall_engage =  merged_data_ow_only.groupby(['Date','Affiliate_Id','Child Offer ID','Hitpath_Offer_ID','Send Strategy'])[['Delivered','Clicks','Optout','Cost','Adjusted Delivered','Adjusted Optout','Adjusted Clicks','Adjusted Cost']].sum().reset_index()
#offerwall_performance = offerwall_engage_subid.merge(offerwall_rev_only, on =['Sub_Id','Child Offer ID'] ,how = 'outer',copy = False )
#offerwall_performance_1 = offerwall_performance.loc[offerwall_performance['Sub_Id'].isna() == False]
#offerwall_performance_2 = offerwall_performance.loc[offerwall_performance['Sub_Id'].isna()== True].drop(columns = ['Sub_Id'])
#offerwall_performance_2 = offerwall_performance_2.groupby( ['Date','Affiliate_Id','Child Offer ID','Hitpath_Offer_ID','Send Strategy'])[[ 'Revenue', 'Jump Page Clicks']].sum().reset_index()
#offerwall_performance_2 = offerwall_performance_2.merge(offerwall_engage,on = ['Date','Affiliate_Id','Child Offer ID','Hitpath_Offer_ID','Send Strategy'] , how ='left')
#offerwall_report = pd.concat([offerwall_performance_1,offerwall_performance_2])
offerwall_rev_only = offerwall_rev_only.groupby(['Date','Affiliate_Id','Child Offer ID','Hitpath_Offer_ID','Send Strategy'])[['Revenue','Jump Page Clicks']].sum().reset_index()
new_ow_df = offerwall_engage.merge(offerwall_rev_only, on = ['Date','Affiliate_Id','Child Offer ID','Hitpath_Offer_ID','Send Strategy'], how = 'outer', copy = False)
new_ow_df = new_ow_df.merge(offer_sheet[['Hitpath Offer ID','Scheduling Name']], copy = False, how = 'left', left_on = 'Child Offer ID', right_on = 'Hitpath Offer ID')
new_ow_df.drop(columns = ['Hitpath Offer ID'], inplace = True)
new_ow_df = new_ow_df.rename(columns = {'Scheduling Name':"Child Offer Campaign Name" })
print(new_ow_df['Revenue'].sum())
print(new_ow_df['Adjusted Delivered'].sum())
merged_data_ow_out =  merged_data_ow.loc[merged_data_ow['Offer Type'] != 'Offer Wall', ]
offerwall_rev_out = offerwall_rev.loc[offerwall_rev['Offer Type']!= 'Offer Wall',]
merged_data_ow_out = pd.concat([merged_data_ow_out,offerwall_rev_out], axis = 0, ignore_index = True) 
new_ow_df['Offer Type'] = 'Offer Wall'
all_combine_df = pd.concat([new_ow_df,merged_data_ow_out], axis = 0, ignore_index = True)
all_combine_df['Hitpath_Offer_ID'] = all_combine_df['Hitpath_Offer_ID'].astype(str).str.split(".",expand = True)[0]
all_combine_df['Affiliate_Id'] = all_combine_df['Affiliate_Id'].astype(str).str.split(".",expand = True)[0]
all_combine_df.rename(columns = {'Hitpath_Offer_ID':'Hitpath Offer ID'}, inplace = True)
all_combine_df['Date'] = pd.to_datetime(all_combine_df['Date'])
all_combine_df['Revenue'] = all_combine_df['Revenue'].fillna(0)
all_combine_df.to_csv(localfolder + "offerwall_merged_all.csv")
#new_ow_df.to_csv(localfolder + "offerwall_merged_all.csv")

#new_ow_df[(new_ow_df['Affiliate_Id']=='461842') & (new_ow_df['Date']=='2024-03-12') & (new_ow_df['Send Strategy']=='P')]
"""
# Group by 'Item' and calculate subtotal to subtract for each item
subtotals_to_subtract = new_ow_df[new_ow_df[ 'Child Offer ID'] != 'Total'].groupby(['Date','Hitpath_Offer_ID','Send Strategy','Affiliate_Id'])[['Revenue','Jump Page Clicks']].sum().reset_index()

# Merge back on the main DataFrame to get the subtraction value per 'Item'
df = pd.merge(new_ow_df, subtotals_to_subtract, on=['Date','Hitpath_Offer_ID','Send Strategy','Affiliate_Id'], how='left', suffixes=('', '_to_subtract'))

# Subtract the subtotals from the Totals
new_ow_df.loc[new_ow_df[ 'Child Offer ID'] == 'Total', 'Revenue'] -= df['Revenue_to_subtract']
#new_ow_df.loc[new_ow_df[ 'Child Offer ID'] == 'Total', 'Conversions'] -= df['Conversions_to_subtract']
new_ow_df.loc[new_ow_df[ 'Child Offer ID'] == 'Total', 'Jump Page Clicks'] -= df['Jump Page Clicks_to_subtract']
# Drop the temporary columns
df.drop(columns=['Revenue_to_subtract','Conversions_to_subtract','Jump Page Clicks_to_subtract'], inplace=True)
"""

7141.0
7141.0


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\218841887.py:38: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data_ow = pd.read_csv(localfolder + 'SS_LC_merged_data.csv')


Offer Wall Revenue 33539.575
Offer Wall Delivered 2330234.0
Offer Wall Revenue 6065.8
Delivery 1760540.0


D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\218841887.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_ow_only.drop(columns = ['Revenue','Jump Page Clicks'], inplace = True)
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_4212\218841887.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Offer List'] = temp['Offer List'].str.replace("[","").str.replace("]","").str.split(",")


6065.799999999999
1760540.0


"\n# Group by 'Item' and calculate subtotal to subtract for each item\nsubtotals_to_subtract = new_ow_df[new_ow_df[ 'Child Offer ID'] != 'Total'].groupby(['Date','Hitpath_Offer_ID','Send Strategy','Affiliate_Id'])[['Revenue','Jump Page Clicks']].sum().reset_index()\n\n# Merge back on the main DataFrame to get the subtraction value per 'Item'\ndf = pd.merge(new_ow_df, subtotals_to_subtract, on=['Date','Hitpath_Offer_ID','Send Strategy','Affiliate_Id'], how='left', suffixes=('', '_to_subtract'))\n\n# Subtract the subtotals from the Totals\nnew_ow_df.loc[new_ow_df[ 'Child Offer ID'] == 'Total', 'Revenue'] -= df['Revenue_to_subtract']\n#new_ow_df.loc[new_ow_df[ 'Child Offer ID'] == 'Total', 'Conversions'] -= df['Conversions_to_subtract']\nnew_ow_df.loc[new_ow_df[ 'Child Offer ID'] == 'Total', 'Jump Page Clicks'] -= df['Jump Page Clicks_to_subtract']\n# Drop the temporary columns\ndf.drop(columns=['Revenue_to_subtract','Conversions_to_subtract','Jump Page Clicks_to_subtract'], inplace=True)

In [27]:
import zipfile
os.chdir(localfolder) 
filename = 'daily_data_update.zip'
email_body = ''
with zipfile.ZipFile('daily_data_update.zip', 'w',zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('SS_LC_merged_data.csv')
toaddr = ['nathan@rxmg.com','lili@rxmg.com','nina@rxmg.com','b.ratzlaff@rxmg.com','n.ohashi@rxmg.com']
#toaddr = ['n.ohashi@rxmg.com','nina@rxmg.com']
#toaddr = ['lili@rxmg.com']
subject_line = 'SMS Daily Data Update' 
email_body 
for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)

connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP d2e1a72fcca58-70673ca69efsm7255092b3a.168 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP d2e1a72fcca58-70673ca69efsm7255092b3a.168 - gsmtp'
connect: b'smtp.gmail.com ESMTP d2e1a72fcca58-70673ca69efsm7255092b3a.168 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [52.38.54.186]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
r

reply: b'354  Go ahead 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp\r\n'
reply: retcode (354); Msg: b'Go ahead 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp'
data: (354, b'Go ahead 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp')
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

reply: b'250 2.0.0 OK  1719423155 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1719423155 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp'
data: (250, b'2.0.0 OK  1719423155 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection 98e67ed59e1d1-2c8d7f5cc20sm1986158a91.17 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connect

## Verification Data Quality 

In [28]:
# check delivered volumn by shortcode name 
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data['Month'] = merged_data['Date'].dt.strftime('%Y-%m')
merged_data[ merged_data['Month'] == '2023-12'].groupby(['Shortcode Name'])[['Delivered']].sum().reset_index()

,Shortcode Name,Delivered
0,DSS,183102.0
1,FLC,2881011.0
2,HZB,1658231.0
3,MBC,3672159.0
4,SVT,897460.0
5,UAA,711621.0
6,UAATF,33008.0
7,W1,0.0


In [29]:
# Check & Examination: 
# Revenue files sum up after November 
rev_file = combined_csv[combined_csv['date']>='2022-11-01'].reset_index(drop=True)['amount'].sum()
# Revenue we made after November
#after_merge = combined_csv_ss_creative_na['Revenue'].sum() + combined_csv_ss_creative_notna['Revenue'].sum() + combined_csv_ss_exclude_11['Revenue'].sum() +flows_clean2['Revenue'].sum() + combined_csv_ss_last_11['amount'].sum() + lc_df_full_11['amount'].sum() + combined_csv_push_11['Revenue'].sum() + combined_csv_w1_11['Revenue'].sum()
after_merge = combined_csv_ss_creative_na['Revenue'].sum() + combined_csv_ss_creative_notna['Revenue'].sum() + combined_csv_ss_exclude_11['Revenue'].sum() +flows_clean2['Revenue'].sum() + combined_csv_ss_last_11['amount'].sum() + combined_csv_push_11['Revenue'].sum() + combined_csv_w1_11['Revenue'].sum()

email_body = "The revenue discrepency after November between raw file and Tableau data is $"+str(round((rev_file - after_merge),2)) + "\n" 
email_body += "The total revenue after November is $" + str(round(rev_file,2)) + "\n" 
email_body += "The revenue discrepency is "+ str( round((rev_file - after_merge) /rev_file,2)*100 ) + "% of the total revenue after November" + "\n" + "The revenue we made after November is $" + str(round(after_merge,2))
print("The revenue discrepency after November between raw file and Tableau data is $",round((rev_file - after_merge),2))
print("The total revenue after November is $", round(rev_file,2))
print("The revenue discrepency is", round((rev_file - after_merge) /rev_file,2)*100, "% of the total revenue after November")

The revenue discrepency after November between raw file and Tableau data is $ -1120.35
The total revenue after November is $ 373751.86
The revenue discrepency is -0.0 % of the total revenue after November


In [30]:
#monthly revenue
merged_data['Month'] = merged_data['Date'].astype(str).str[:7]
temp = merged_data.groupby('Month')['Revenue'].sum()
email_body +=  "\n\nThe revenue by month is\n" 
email_body += str(temp)  + '\n'
temp

Month
1899-12         0.000000
1999-12       652.800000
2019-12         0.000000
2020-01         0.000000
2020-02       163.100000
2020-03      1116.300000
2020-04      5060.200000
2020-05      2218.760000
2020-06      3551.800000
2020-07      6578.100000
2020-08     41870.600000
2020-09     52336.370000
2020-10     50779.900000
2020-11     37785.730000
2020-12     32358.100000
2021-01     23948.040000
2021-02     20137.840000
2021-03     26584.280000
2021-04     50262.800000
2021-05     74756.520000
2021-06     91146.100000
2021-07    144141.810000
2021-08    158429.500000
2021-09    154983.178800
2021-10    201033.211100
2021-11    225191.850000
2021-12    266562.230000
2022-01    276934.304528
2022-02    268022.813635
2022-03    249501.265543
2022-04    296194.537417
2022-05    277401.288745
2022-06    404793.833177
2022-07    247403.937505
2022-08    155207.055008
2022-09    161623.944255
2022-10     39895.442484
2022-11     87340.753168
2022-12    112803.439432
2023-01    126387.6

In [31]:
combined_csv_ss_last_11['month'] = combined_csv_ss_last_11['date'].astype(str).str[:7]

email_body += "\n" + "The revenue by month for unclaimed revenue is " + str(combined_csv_ss_last_11.groupby('month')['amount'].sum()) +"\n"
combined_csv_ss_last_11.groupby('month')['amount'].sum()

month
2024-04     10.32
2024-05    301.96
2024-06    130.79
Name: amount, dtype: float64

In [32]:
temp = merged_data.groupby('Date')['Revenue'].sum()
email_body +=  "\n\nThe revenue by Date is\n" 
email_body += str(temp)  + '\n' 
temp

Date
1899-12-31       0.00000
1999-12-31     652.80000
2019-12-25       0.00000
2019-12-26       0.00000
2020-01-24       0.00000
                 ...    
2024-06-22    5839.34889
2024-06-23    6438.76424
2024-06-24    8648.00650
2024-06-25    6213.65700
2024-06-26     267.84000
Name: Revenue, Length: 1590, dtype: float64

# Send out email to the Team 

In [33]:
toaddr = ['lili@rxmg.com','nathan@rxmg.com','nina@rxmg.com','n.ohashi@rxmg.com']
#toaddr = ['nina@rxmg.com']
#toaddr = ['lili@rxmg.com']
subject_line = 'SMS Daily Update Report' 
email_body 
for i in toaddr:
    send_email.send_email('',subject_line,email_body,i)

connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP 41be03b00d2f7-716ba6aa524sm8902730a12.63 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP 41be03b00d2f7-716ba6aa524sm8902730a12.63 - gsmtp'
connect: b'smtp.gmail.com ESMTP 41be03b00d2f7-716ba6aa524sm8902730a12.63 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [52.38.54.186]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
repl

reply: b'250 2.0.0 OK  1719423266 98e67ed59e1d1-2c8d7f5a6aesm1937675a91.22 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1719423266 98e67ed59e1d1-2c8d7f5a6aesm1937675a91.22 - gsmtp'
data: (250, b'2.0.0 OK  1719423266 98e67ed59e1d1-2c8d7f5a6aesm1937675a91.22 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection 98e67ed59e1d1-2c8d7f5a6aesm1937675a91.22 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection 98e67ed59e1d1-2c8d7f5a6aesm1937675a91.22 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP d2e1a72fcca58-706863876ebsm5951814b3a.142 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP d2e1a72fcca58-706863876ebsm5951814b3a.142 - gsmtp'
connect: b'smtp.gmail.com ESMTP d2e1a72fcca58-706863876ebsm5951814b3a.142 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENH

reply: b'354  Go ahead 41be03b00d2f7-716b4a73051sm9023758a12.47 - gsmtp\r\n'
reply: retcode (354); Msg: b'Go ahead 41be03b00d2f7-716b4a73051sm9023758a12.47 - gsmtp'
data: (354, b'Go ahead 41be03b00d2f7-716b4a73051sm9023758a12.47 - gsmtp')
send: b'Content-Type: multipart/mixed; boundary="===============2094581291733028325=="\r\nMIME-Version: 1.0\r\nFrom: schedulealartsrxmg@gmail.com\r\nTo: n.ohashi@rxmg.com\r\nSubject: SMS Daily Update Report\r\n\r\n--===============2094581291733028325==\r\nContent-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\n\r\nThe revenue discrepency after November between raw file and Tableau data is $-1120.35\r\nThe total revenue after November is $373751.86\r\nThe revenue discrepency is -0.0% of the total revenue after November\r\nThe revenue we made after November is $374872.22\r\n\r\nThe revenue by month is\r\nMonth\r\n1899-12         0.000000\r\n1999-12       652.800000\r\n2019-12         0.000000\r\n2020-01   

In [34]:
end_time = time.time()

In [35]:
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 1632.72 seconds
